# Parse development book

### Notebook purpose
This notebook is development space for python parse.ts replacement and upgrade.
It reads specified google sheets and output actants.json file, which can be imported to inkVisitor RethinkDB.py

The aggreated doc for dev: https://docs.google.com/document/d/1ga6R_9TWAQlXE9XqPE_qZ2S8S8aVW2A7n8SuYpSXnoI/edit#heading=h.q9ntf0ofam2u
The holy schema: https://app.diagrams.net/#G1bKvqEKr6JzPWryVg-vYudQy_4KvuzHS9

### JSon schemas for the actants
 * are created from *.ts files through typescript-jschema commandline utility
 * the schemas are trasformed as python Classes through warlock library


### Principles of the parsing operations
 * There are google sheet dataset, which need to be transformed to json format according to inkVisior datamodel
 * inkVisitor holds datamodel in the code, the typescript classes (see /shared/types...), or the holy schema
 * DZ created in input table parsing instructions
   * the first 5 rows contain instructions
   * if the keywords contain ? character, they are ignored (it is work in progress from DZ)

#### Parsing instructions
 * explicit set of keywords
   * discard
     * ...
   * inside
     * the value in the column should be straitforwadly made part of the entity object, the exists update_generic method or update_fieldName mehthods for fields with custom but generally applicable logic
   * propvalue
     * for making so called
          * statement or property statment (old A0093 has relation)
     * it sits in the entity "props" attribute, it has IProp class
   * special
     * fully custom method for the parsing behavior
     *

### Parsing process
 1. Prepare data model entities classes (from typescript inkvisitor classes -> json schema ->  python classes)
 2. Load and wrangle all input data
 3. Process headers of the tables for parsing instructions
 4. Process tables row by row, column by column
 5. Save as json file (which can be imported to the inkVisitor RethinkDB)


# DEV Updates : November
 - TODO updated model for relations (attribute order)
 - TODO synonymic relations parsed to "cloud"

# Dev updates:  October 2
- DONE change event-territories generation,  genre is relation.classification, not metaprop
- TODO prepare for "partial parsing" and fixed guids for "done" tables
- DONE updated users
- DONE change in Manuscript parsing, For each non-empty, non-NA value: check whether the script hasn't already generated an L entity with the same label.
IF NOT: (1) Generate a new L entity with label = value in this col., status = "approved", entity logical type = "definite", label language = value in the next col. (repository_label_language); (2) append to this L entity a RelationType.Classification leading to C2646 "archive or library". (3) Go to No. 4.
IF YES, go to No. 4 and use that L.
(4) Under the O entity representing the row (i.e. the physical manuscript), add a metaprop which will relate this O to this L entity (repository) through the relation: O - (has) - C2645 "repository" - L in this col.




# Dev updates:  October
 - DONE u action a conceptu se neparsuji relations ?
 - DONE revize skriptu vzhledem k september bordelu
 - DONE events parsing
 - DONE 21.09. 2022, R table has received a new parsable column “components_id”. Its notation follows our agreed-upon notation, so it should not pose any problem, but I rather signal this new col. to you. It is for composite Rs, which need to be separated into several identities with different URIs (typically two volumes of an edition)
  - DONE The instruction here and here for generating creation events for Territories and manuscript Objects has changed, since what was the Property C0565 class is now the RelationType Classification. Please take this change into account, tx.
 - DONE territories generation
  - light yellow: metadata of Subterritory capturing the lowest-level document recording the Event (not of Event described by the row). Tomáš H.: (1) Please generate these lowest-level T entities following Adam’s script for parsing Robert’s Sellan coding sheet IDs into a hierarchical T structure and put them under their proper root T through legacy IDs: first element of ID = root T ID, last element = lowest-level Territory whose label follows in the next col., middle elements = folders in between root and lowest child level. E.g. here, in Guglielmites, there is root, i.e. the whole Guglielmites process of Beltramo Salvagno; then parts 1-4; and underneath them, the individual documents. When creating these lowest-level Subterritories, use this attributes: Entity type = T; T label and label_language defined in the two respective cols. which follow; status = approved. Other metadata set as metaprops in the following yellow cols. (2) Generate the middle entities holding the hierarchy, name them by the chunk ID (in this case, this will be four Ts named “1”, “2”, “3”, and “4”), and relate the lowest-level entity to them as their parents, and relate those middle entities to the root T as their parent in turn; attributes of these middle entities (folders 1-4 of the records): label = number [1,2,3,4] depending on the chunk; legacy id = T107-[chunk number], label language = Latin, status = approved. (3) Relate the lowest-level entity to the Event described by the row by the following Metaproperty: T107-1-01 - [has] - C2286 represented event - E0001 (middle-level entities will have no such relation now, only lowest-level documents).


# Dev updates:  September
 - adding new users

# Dev updates:  August
 - new instruction with proptype_2nd in column
    - before we got the type fixed and had just instruction propvalue_2nd (used in territories for manuscript witness)
 - new instruction "hook-inside" for insides in newly created entities (see in gugli locations special:modern_name)
 - changes :  some properties 'id' changed to 'specificId'
   - IProp {... 'value':{'entityId' and 'type':{'entityId'
   - IStatement {  'data':{ 'territory' {'territoryId'
   - ITerritory { 'data': {'parent': {'territoryId': ""
- bigger TODO relations

# Dev updates: April

## Concepts
  * DONE wordnet_synset_id
    * wordnet_id bounded to resource object R0067

## Texts
 * DONE special_creation_event_id
   * instructions:
     * Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: see next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to any of those Es the metaprop "(has) - C0565 "class" - C2642 "creation" (to instantiate the event to its event type = event class).
   * creation_event_label in standalone field
     * this will trigger the operation
   * lot of proptype and propvalue with non-standard schema: new branch of interpreting

## Manuscripts
 * special_repository_label
   * i:
     * For each non-empty, non-NA, non-NS row: (1) generate L entity with label = value in this col., status = "approved", entity logical type = "definite", label language = value in the next col. (repository_label_language); (2) append to this L entity a metaprop (has) - C0565 "class" - C2646 "archive or library", and (3) under the O entity representing the row (i.e. the physical manuscript), add a metaprop which will relate this O to this L entity (repository) through the relation: O - (has) - C2645 "repository" - L in this col.
   * repository_label_language
     * i:
       * Use this value as label language value of the repository L entity.
 * DONE creation_event_id
   * creation_event_label
   * i:
     * Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: see next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to any of those Es the metaprop "(has) - C0565 "class" - C2642 "creation" (to instantiate the event to its event type = event class).
 * reproduction_online_url
   * instructions:
     * If non-empty, non-NA, (1) generate an R entity with label "Reproduction of " + label of the MS (i.e. value in the B column, status = "approved", label-language = "English", url = the URL sitting under the hyperlink value in this cell, and (2) add metaprop to the O entity represented by this row: O - (has) - C1199 "digital reproduction" - the R entity here generated.

### Input variables

In [1]:
input_tables = ["texts", "manuscripts", "resources", "actions" , "concepts"]

#                   sheet_name,  gdrive code, header_in_row, columns which need suffix based on a case name
input_sheets = {
    "texts" : ("Texts","13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5), #https://docs.google.com/spreadsheets/d/13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE/edit#gid=2056508047
    "manuscripts" : ("Manuscripts", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "resources" : ("Resources", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "actions" :  ("Statements","1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ", 5), # https://docs.google.com/spreadsheets/d/1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ/edit#gid=0
    "concepts" : ("Concepts","1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4", 5), # https://docs.google.com/spreadsheets/d/1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4/edit#gid=0,
    "R0006_persons" : ("Persons","1UtgKvjr91egdgOQMP8WxsRxgGDT9FC92qBG3sKBiYvk", 5, ['id','residence_id','social_relation1_value_id','social_relation2_value_id','social_relation3_value_id']), # GUGLIELMITES Persons, https://docs.google.com/spreadsheets/d/1UtgKvjr91egdgOQMP8WxsRxgGDT9FC92qBG3sKBiYvk/edit#gid=0,
    "R0007_locations" : ("Locations","1jImuYCTu-QrZc7oCT1Vz01SLu5sxYRo4xzJzlF2J8Bs", 5, ['id','locationrelation1_value_id','superordinate_location_id']) ,# GULIELMITES Locations, https://docs.google.com/spreadsheets/d/1jImuYCTu-QrZc7oCT1Vz01SLu5sxYRo4xzJzlF2J8Bs/edit#gid=0,
    "R0008_events" : ("Events","1mLevawnoZVLQv6qINaSfiyi1NNlfJzTZevinRSSQVPo", 5, ['id','spatial_ref_value_id','participant_id','inquisitor_id','notary_id','witness_assessor_id','accuser_id','accused_id','mentioned_id','denied_id','marked_as_incriminated_id','temporal_ref_1_value_id','temporal_ref_2_value_id']),
    # ORIGINAl https://docs.google.com/spreadsheets/d/1mLevawnoZVLQv6qINaSfiyi1NNlfJzTZevinRSSQVPo/edit#gid=0,
    "R0075_persons" : ("Persons","1Tyi4DYnHDeTs8tuz9rxN3wyq1LqLWWp_ZPgOimQoxf8", 5, ['id','origin_or_residence_id','residence_id','relations_1_value_id','relations_2_value_id','relations_3_value_id','relations_4_value_id','relations_5_value_id','relations_6_value_id','relations_7_value_id','relations_8_value_id','relations_9_value_id','relations_10_value_id','relations_11_value_id','relations_12_value_id','relations_13_value_id','origin_id','occupation_id','office_id']), # Castellario Persons, https://docs.google.com/spreadsheets/u/1/d/1Tyi4DYnHDeTs8tuz9rxN3wyq1LqLWWp_ZPgOimQoxf8/edit#gid=0
    "R0035_locations" : ("Locations","1VwUDnAMJ8mWvL-54K-OnZmguB-rALnJjtGG2IGNBqIU", 5, ['id','locationrelation1_value_id','locationrelation2_value_id']) ,# Piedmont Castellario Locations, https://docs.google.com/spreadsheets/d/1VwUDnAMJ8mWvL-54K-OnZmguB-rALnJjtGG2IGNBqIU/edit#gid=0
    "R0083_events" : ("Events","1M5uhJsbjUCxWP_avYQM0fr1Hm0GXVmYuDvqSoEuUAbs", 5, ['id','spatial_ref_value_id','participant_id','inquisitor_id','notary_id','witness_assessor_id','accuser_id','accused_id','mentioned_id','denied_id','temporal_ref_1_value_id','temporal_ref_2_value_id'])
    # Castellario Incriminations https://docs.google.com/spreadsheets/u/1/d/1M5uhJsbjUCxWP_avYQM0fr1Hm0GXVmYuDvqSoEuUAbs/edit#gid=0
}

# resource IDs
# R0006 Guglielmites Persons
# R0007	Guglielmites Locations
# R0008	Guglielmites Events

# R0035	Piedmont Locations Castellario
# R0075	Persons Castellario
# R0083 Castellario Incriminations

table_to_entity = {
    "concepts" : "IConcept",
    "resources" : "IResource",
    "texts" : "ITerritory",
    "manuscripts" : "IObject",
    "actions" :  "IAction",
    "R0006_persons" : "IPerson",
    "R0007_locations": "ILocation",
    "R0008_events": "IEvent",
    "R0075_persons" : "IPerson",
    "R0035_locations": "ILocation",
    "R0083_events": "IEvent",
}

root_sheet_url = "https://docs.google.com/spreadsheets/d/"
google_api_dotenv_path = "../env/.env.googleapi"  # contains google api specs for sheet access with Dator
schema_path = '../schemas/' # path for dir with schemas
json_schemas = {}  # holder for schemas, so they can be used for jsonschema validate

In [3]:
import subprocess
subprocess.run("python generate-json-schemas.py", shell=True,capture_output=True)

CompletedProcess(args='python generate-json-schemas.py', returncode=0, stdout=b'..\\node_modules\\.bin\\ts-json-schema-generator --path ../../shared/types/actant.ts IActant --tsconfig ../tsconfig-es2015.json --type IActant --out ../schemas/IActant.schema.json\r\n..\\node_modules\\.bin\\ts-json-schema-generator --path ../../shared/types/concept.ts IConcept --tsconfig ../tsconfig-es2015.json --type IConcept --out ../schemas/IConcept.schema.json\r\n..\\node_modules\\.bin\\ts-json-schema-generator --path ../../shared/types/object.ts IObject --tsconfig ../tsconfig-es2015.json --type IObject --out ../schemas/IObject.schema.json\r\n..\\node_modules\\.bin\\ts-json-schema-generator --path ../../shared/types/event.ts IEvent --tsconfig ../tsconfig-es2015.json --type IEvent --out ../schemas/IEvent.schema.json\r\n..\\node_modules\\.bin\\ts-json-schema-generator --path ../../shared/types/entity.ts IEntity --tsconfig ../tsconfig-es2015.json --type IEntity --out ../schemas/IEntity.schema.json\r\n..\\n

### Libraries

In [2]:
import os, warlock, json
from datetime import datetime
import time
from jsonschema import validate
import dissinetpytools.dator as dator
from dotenv import load_dotenv

import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.filterwarnings("ignore")

import numpy as np
from copy import deepcopy
from shutil import copyfile

import uuid

def get_uuid_id():
    return str(uuid.uuid4())

def is_valid_uuid(val):
    try:
        uuid.UUID(str(val))
        return True
    except ValueError:
        return False

# type hinting
from collections.abc import Sequence, Callable
from typing import List, Dict, Tuple


### Initialisation

In [3]:
load_dotenv(google_api_dotenv_path) # fills os.environ['GDRIVE_API_CREDENTIALS']
d = dator.Dator(loglevel=10, print_log_online=True, cache=True, project_name="inkvisitor-import") # expects 'GDRIVE_API_CREDENTIALS' in the global system variables (os.environ)
d.google_authenticate()
logger = d.logger

20 2022-11-25 14:26:02 : Google authentification start
20 2022-11-25 14:26:02 : Google authentification end
20 2022-11-25 14:26:02 : Dator initiation succesfull end


In [4]:
# read all schemas inside and warlock them as globally available classes
schema_filenames = os.listdir(schema_path)
json_classes = {}
for schema in schema_filenames:
    name = schema.split(".schema")[0]
    file_handler = open(schema_path + schema,"r")
    schema_json = json.load(file_handler)
    json_schemas[name] = schema_json
    globals()[name] = warlock.model_factory(schema_json)
    json_classes[name] = globals()[name]
    logger.info("Class " + name + " available.")

logger.info(f"There are {len(json_classes.keys())} json classes available ({' '.join(json_classes.keys())}).")

2022-11-25 14:26:04,673 INFO Class IActant available.
2022-11-25 14:26:04,677 INFO Class IAction available.
2022-11-25 14:26:04,681 INFO Class IAudit available.
2022-11-25 14:26:04,685 INFO Class IConcept available.
2022-11-25 14:26:04,689 INFO Class IEntity available.
2022-11-25 14:26:04,692 INFO Class IEvent available.
2022-11-25 14:26:04,695 INFO Class ILabel available.
2022-11-25 14:26:04,699 INFO Class ILocation available.
2022-11-25 14:26:04,703 INFO Class IObject available.
2022-11-25 14:26:04,707 INFO Class IPerson available.
2022-11-25 14:26:04,711 INFO Class IProp available.
2022-11-25 14:26:04,714 INFO Class IReference available.
2022-11-25 14:26:04,717 INFO Class IResource available.
2022-11-25 14:26:04,720 INFO Class IStatement available.
2022-11-25 14:26:04,724 INFO Class ITerritory available.
2022-11-25 14:26:04,727 INFO Class IUser available.
2022-11-25 14:26:04,731 INFO Class IValue available.
2022-11-25 14:26:04,734 INFO Class Relation.IActant1Semantics available.
202

In [5]:
# factory for making entity objects, contains defaults with "prerequisities"
from datetime import datetime
import_note = "Import batch [development] " + str(datetime.now())

class InkVisitorJSONObjectFactory:

    classes = json_classes

    json_class_defaults = {
        'IAction':{
            'class':'A', 'id':'','legacyId':'', 'label':'', 'language':'', 'detail':'','data':{'entities':{'a1':[],'a2':[],'s':[]},'valencies':{'a1':'','a2':'','s':''}}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IConcept':{
            'class':'C', 'id':'', 'legacyId':'','label':'', 'language':'', 'detail':'','data':{}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IValue':{
            'class':'V', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IProp':{
            'bundleEnd':False,'bundleStart':False, 'certainty':'0', 'children':[], 'elvl':'3', 'id':'', 'logic':'1', 'mood':['1'], 'moodvariant':'1', 'bundleOperator':'a', 'type': {'entityId':'','elvl':'3','logic':'1','partitivity':'1','virtuality':'1'},'value':{'entityId':'','elvl':'3', 'logic':'1', 'partitivity':'1', 'virtuality':'1'}
        },
        'IResource':{
             'class':'R', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'url':'','partValueBaseURL':'','partValueLabel':''}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'Relation.IIdentification':{
            'id':'', 'type':'IDE', 'certainty':'0','entityIds': ['','']
        },
        'Relation.IClassification':{
            'id':'', 'type':'CLA','entityIds': ['',''],"order":1
        },
        'Relation.IImplication':{
            'id':'', 'type':'IMP','entityIds': ['',''], "order":1
        },
        'Relation.IHolonym':{
            'id':'', 'type':'HOL','entityIds': ['',''],"order":1
        },
        'Relation.IRelated':{
            'id':'', 'type':'REL','entityIds': ['',''], "order":1
        },
        'Relation.IActionEventEquivalent':{
            'id':'', 'type':'AEE','entityIds': ['','']
        },
        'Relation.ISubjectActant1Reciprocal':{
            'id':'', 'type':'SAR','entityIds': ['','']
        },
        'Relation.IPropertyReciprocal':{
            'id':'', 'type':'PRR','entityIds': ['','']
        },
        'Relation.IAntonym':{
            'id':'', 'type':'ANT','entityIds': ['',''], "order":1
        },
        'Relation.ISynonym':{
            'id':'', 'type':'SYN','entityIds': ['','']
        },
        'Relation.ISuperordinateLocation':{
            'id':'', 'type':'SOL','entityIds': ['',''], "order":1
        },
        'Relation.ISuperclass':{
            'id':'', 'type':'SCL','entityIds': ['',''], "order":1
        },
        'Relation.ISubjectSemantics':{
            'id':'', 'type':'SUS','entityIds': ['',''], "order":1
        },
        'Relation.IActant1Semantics':{
            'id':'', 'type':'A1S','entityIds': ['',''], "order":1
        },
        'Relation.IActant2Semantics':{
            'id':'', 'type':'A2S','entityIds': ['',''], "order":1
        },
        'IObject':{
             'class':'O', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IStatement':{
             'class':'S', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'actants':[], 'actions':[], 'tags':[],'territory': {'territoryId':'','order':0}, 'text':''}, 'props':[], 'notes':[], 'status':'1','references':[]
        },
        'ITerritory':{
            'class':'T', 'id':'', 'legacyId':'','label':'', 'language':'', 'detail':'','data':{'parent':{ "territoryId": "T0", "order": 0 }}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'ILocation':{
            'class':'L', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IEvent':{
            'class':'E', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IReference':{
           'id':'','resource':'','value':''
        },
        'IAudit':{
           'id':'','entityId':'','user':'','date':'','changes':{}
        },
        'IPerson':{
            'class':'P', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
    }

    def __init__(self):
        for key, item in type(self).json_class_defaults.items():
            if 'notes' in item and  len(item['notes']) == 0:
                item['notes'].append(import_note)

    def make(self, entity_name, override_object=None):
        if override_object is None:
            override_object = {}
        object = type(self).json_class_defaults[entity_name]
        object.update(override_object)
        return type(self).classes[entity_name](deepcopy(object))

    def validate_defaults(self):
        for e in self.json_class_defaults:
            d.logger.info(f"Trying to validate class {e}.")
            test = self.make(e, self.json_class_defaults[e])
            d.logger.info(f"Class {e} validated.")


IOF = InkVisitorJSONObjectFactory()
IOF.validate_defaults()

2022-11-25 14:26:07,905 INFO Trying to validate class IAction.
2022-11-25 14:26:07,908 INFO Class IAction validated.
2022-11-25 14:26:07,908 INFO Trying to validate class IConcept.
2022-11-25 14:26:07,911 INFO Class IConcept validated.
2022-11-25 14:26:07,911 INFO Trying to validate class IValue.
2022-11-25 14:26:07,914 INFO Class IValue validated.
2022-11-25 14:26:07,914 INFO Trying to validate class IProp.
2022-11-25 14:26:07,916 INFO Class IProp validated.
2022-11-25 14:26:07,916 INFO Trying to validate class IResource.
2022-11-25 14:26:07,919 INFO Class IResource validated.
2022-11-25 14:26:07,920 INFO Trying to validate class Relation.IIdentification.
2022-11-25 14:26:07,921 INFO Class Relation.IIdentification validated.
2022-11-25 14:26:07,921 INFO Trying to validate class Relation.IClassification.
2022-11-25 14:26:07,922 INFO Class Relation.IClassification validated.
2022-11-25 14:26:07,923 INFO Trying to validate class Relation.IImplication.
2022-11-25 14:26:07,924 INFO Class R

In [11]:
# initialisation of input data tables
tables = {}
header_infos = {}
entity_ids = {}

# Load input datasets

In [42]:
# partial load
# partial_load = ['R0075_persons', 'R0035_locations','R0083_events']
# partial_load = ['R0035_locations']
partial_load = []

# empty value unifier
def unify_empty_value(df: pd.DataFrame, empty_values=None, unified_empty_value =''):
    if empty_values is None:
        empty_values = ["#N/A", "#VALUE!",'NA', 'NS'] # 'NA', 'NS',
    for naner in empty_values:
        df = df.replace(naner,unified_empty_value, regex=True)
    df. fillna(unified_empty_value, inplace=True)
    return df

# define cross-suffixes
casearea_to_entity_suffix = {
    "R0006" : {
        "P" : "R0006",
        "L" : "R0007",
        "E" : "R0008"
    },
    "R0007" : {
        "P" : "R0006",
        "L" : "R0007",
        "E" : "R0008"
    },
    "R0008" : {
        "P" : "R0006",
        "L" : "R0007",
        "E" : "R0008"
    },
    "R0075" : {
        "P" : "R0075",
        "L" : "R0035",
        "E" : "R0083"
    },
    "R0035" : {
        "P" : "R0075",
        "L" : "R0035",
        "E" : "R0083"
    },
    "R0083" : {
        "P" : "R0075",
        "L" : "R0035",
        "E" : "R0083"
    },
}

def get_suffix_based_on_case_and_entity_type(string, case_suffix):
    # if valid entity type
    allowed_strict_entities = ['P','L','E','B']
    if any(string.startswith(c)for c in allowed_strict_entities):
        fl = string[0]
        return "_"+casearea_to_entity_suffix[case_suffix.replace("_","")][fl]
    else:
        return ""

def apply_suffix(string, case_suffix):
    if string in ['NS','NA']:
        return string
    mod_string = ""
    if "#" in string:
        strings = string.split("#")
        for s in strings:
            mod_string += " #"+ s.strip() + get_suffix_based_on_case_and_entity_type(s.strip(), case_suffix)
        mod_string = mod_string[2:]
    else:
        mod_string = string + get_suffix_based_on_case_and_entity_type(string, case_suffix)
    return mod_string


# load all input tables
load_sheets = {}
if len(partial_load) > 0:
    logger.info(f"Making PARTIAL LOAD with {partial_load}")
    for item in partial_load:
        load_sheets[item] = input_sheets[item]
else:
    load_sheets = input_sheets

for key, sheet in load_sheets.items():
    logger.info(f"Calling for {key} with sheet_name {sheet[0]}.")
    tables[key], header_infos[key] = d.load_df_from_gsheet(key,root_sheet_url + sheet[1], sheet[0], fromCache=False, header_in_row=sheet[2], clean=True, fillna=True, cleanByColumn="label", parse_hyperlink_formulas=True, numerize=False)
    tables[key] = unify_empty_value(tables[key])
    header_infos[key] = unify_empty_value(header_infos[key])

    # clean brackets
    tables[key] = tables[key].replace("\[","",  regex=True)
    tables[key] = tables[key].replace("\]","",  regex=True)


    # case study dataset legacyId needs a suffix
    if "_" in key:
        columns = sheet[3]
        suffix = "_" + key.split("_")[0]
        for c in columns:
            # tables[key][c] = tables[key][c] + suffix
            logger.info(f"Using {key} and {c}.")
            tables[key][c]  = tables[key][c].apply(lambda item: apply_suffix(item,suffix) if len(str(item)) > 0 else item)

            #tables[key].loc[~tables[key][c].isna(), "c"] = tables[key][c] + "_" + suffix
            # df.loc[df["gender"] == "male", "gender"] = 1
            logger.info(f"{key} For columns {c} the suffix from {suffix} set was glued.")

    # code for legacyId copy and uuid creation
    # tables[key]['legacyId'] = tables[key]['id'].copy()
    tables[key].insert(2, 'legacyId', tables[key]['id'].copy())

    # inform instructive header about the new column and what to do with it
    header_infos[key]['legacyId'] = ""
    header_infos[key].at[3,'legacyId'] = "inside"
    tables[key]['id'] = tables[key].apply(lambda x: get_uuid_id(), axis=1)  # generate unique id for each row
    # make id dictionaries (it is much faster to search for keys there in legacyId>id retrievals)
    ed = tables[key][["legacyId","id"]].set_index("legacyId")

    if "_" in key:   # legacyIds are case-specified, i.e. all locations can be together in one table
        key = key.split("_")[1]

    if key in  entity_ids:
        entity_ids[key] = {**entity_ids[key],**ed["id"].to_dict()}
    else:
        entity_ids[key] = ed["id"].to_dict()

2022-11-25 19:14:49,050 INFO Calling for texts with sheet_name Texts.


20 2022-11-25 19:14:50 : Loading dataset texts
20 2022-11-25 19:14:50 : Opting for variant header at row 5.
20 2022-11-25 19:15:08 : Hyperlinks were detected and transformed in columns ['edition_1', 'edition_2', 'edition_3', 'persons_index_link', 'places_index_link'].
20 2022-11-25 19:15:08 : Dropping empty columns in the dataset texts : (1011, 115)
20 2022-11-25 19:15:08 : Deleted 859 empty rows by label.
20 2022-11-25 19:15:08 : Loaded and prepared dataset texts : (152, 115)
20 2022-11-25 19:15:08 : Making pickle cache of  texts with separeted header file : (152, 115)


2022-11-25 19:15:09,064 INFO Calling for manuscripts with sheet_name Manuscripts.


20 2022-11-25 19:15:09 : Loading dataset manuscripts
20 2022-11-25 19:15:09 : Opting for variant header at row 5.


2022-11-25 19:15:11,501 INFO Calling for resources with sheet_name Resources.


...

2022-11-25 19:16:37,508 INFO Using R0035_locations and id.
2022-11-25 19:16:37,509 INFO R0035_locations For columns id the suffix from _R0035 set was glued.
2022-11-25 19:16:37,510 INFO Using R0035_locations and locationrelation1_value_id.
2022-11-25 19:16:37,511 INFO R0035_locations For columns locationrelation1_value_id the suffix from _R0035 set was glued.
2022-11-25 19:16:37,511 INFO Using R0035_locations and locationrelation2_value_id.
2022-11-25 19:16:37,512 INFO R0035_locations For columns locationrelation2_value_id the suffix from _R0035 set was glued.
2022-11-25 19:16:37,516 INFO Calling for R0083_events with sheet_name Events.


20 2022-11-25 19:16:37 : Hyperlinks were detected and transformed in columns [].
20 2022-11-25 19:16:37 : Dropping empty columns in the dataset R0035_locations : (1000, 31)
20 2022-11-25 19:16:37 : Deleted 845 empty rows by label.
20 2022-11-25 19:16:37 : Loaded and prepared dataset R0035_locations : (155, 31)
20 2022-11-25 19:16:37 : Making pickle cache of  R0035_locations with separeted header file : (155, 31)
20 2022-11-25 19:16:38 : Loading dataset R0083_events
20 2022-11-25 19:16:38 : Opting for variant header at row 5.


2022-11-25 19:16:56,925 INFO Using R0083_events and id.


20 2022-11-25 19:16:56 : Hyperlinks were detected and transformed in columns [].
20 2022-11-25 19:16:56 : Dropping empty columns in the dataset R0083_events : (1622, 57)
20 2022-11-25 19:16:56 : Deleted 1374 empty rows by label.
20 2022-11-25 19:16:56 : Loaded and prepared dataset R0083_events : (248, 57)
20 2022-11-25 19:16:56 : Making pickle cache of  R0083_events with separeted header file : (248, 57)


2022-11-25 19:16:56,926 INFO R0083_events For columns id the suffix from _R0083 set was glued.
2022-11-25 19:16:56,927 INFO Using R0083_events and spatial_ref_value_id.
2022-11-25 19:16:56,928 INFO R0083_events For columns spatial_ref_value_id the suffix from _R0083 set was glued.
2022-11-25 19:16:56,928 INFO Using R0083_events and participant_id.
2022-11-25 19:16:56,929 INFO R0083_events For columns participant_id the suffix from _R0083 set was glued.
2022-11-25 19:16:56,930 INFO Using R0083_events and inquisitor_id.
2022-11-25 19:16:56,931 INFO R0083_events For columns inquisitor_id the suffix from _R0083 set was glued.
2022-11-25 19:16:56,932 INFO Using R0083_events and notary_id.
2022-11-25 19:16:56,933 INFO R0083_events For columns notary_id the suffix from _R0083 set was glued.
2022-11-25 19:16:56,933 INFO Using R0083_events and witness_assessor_id.
2022-11-25 19:16:56,934 INFO R0083_events For columns witness_assessor_id the suffix from _R0083 set was glued.
2022-11-25 19:16:56,

In [8]:
# making empty table of values
tables['values'] = pd.DataFrame(columns=['id','value','origin'])
tables['locations'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['events'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['props'] = pd.DataFrame(columns=['id','type','type_id','value','value_id','original_field','origin','entityId','legacyId'])
tables['relations'] = pd.DataFrame(columns=['id','type','logic','certainty','entityIds','origin'])

In [9]:
#tables["R0008_events"] = tables["R0008_events"].replace("\[","",  regex=True)
#tables["R0008_events"] = tables["R0008_events"].replace("\]","",  regex=True)

df = tables["concepts"]
df[df['label']=='social category']

,status,id,legacyId,label,detail,dissinet_checker,checked,label_language,wordnet_lemma_id,wordnet_synset_id,...,note,recommendation,editor,statement_template,core_relations_actions,core_relations_concepts,core_relations_PLOGE,core_relation_h5_agreement,DEPRECATED_label_english,parsing_rows_explained
1660,approved,ebac116f-8343-4f77-add6-b09df89761d3,C0331,social category,"i.e. the category itself, not its holder",DS,y,English,,,...,"The superclass of this Concept is ""class"", sin...","For things such as hereticus perfectus, faydit...",Davor Salihović,,,,,,social category,


In [10]:
df[df['label']=='occupation']['id']

1369    9cf803bb-a853-455c-88d0-425b0d81a568
Name: id, dtype: object

In [11]:
entity_ids['locations']

{'L0001_R0007': '7f49cd3c-d5f8-4331-b798-3a56f682f60d',
 'L0002_R0007': 'fb00d80f-05a4-4501-baff-9bace476f0bf',
 'L0003_R0007': 'b4745cef-9040-4def-82b2-7bb33a8f4829',
 'L0004_R0007': 'bf0a9f8e-c913-4931-928a-5399f05d45bb',
 'L0005_R0007': '05b77d65-3dc4-4df4-b73f-00c58356512b',
 'L0006_R0007': 'f5bfab72-d951-4155-a65e-96549f710f2f',
 'L0052_R0007': '61d2e2b9-4d28-442a-9315-b384ecec9a07',
 'L0053_R0007': '1626f63d-f0ce-414d-ad3f-b3f07bb6a8d9',
 'L0051_R0007': 'cd799a2d-c6b4-48f6-9b7d-13f56bbcdbfa',
 'L0007_R0007': '36742b02-da64-4c1e-9c29-ec329cc565b9',
 'L0008_R0007': '3c344880-2eb5-4911-8b5d-d4cc49b89b55',
 'L0009_R0007': '8e1622ff-a2be-460c-9f8f-1364c47539b3',
 'L0010_R0007': '39a967ac-9d60-4dcc-b55a-6b3d078ace25',
 'L0011_R0007': '6e8d884c-c879-4615-9619-2995bb761661',
 'L0012_R0007': '3cfe5344-e608-4c3e-84e1-f4105e0796f9',
 'L0013_R0007': 'e9685976-163b-4464-8de5-d26327f4c259',
 'L0014_R0007': 'd5a8681d-0125-4f67-9935-6c07a6cb632e',
 'L0015_R0007': '01bba9ad-59f0-47a9-99cc-f6a39e2

In [20]:
# manual corrections in the input data

### Responsible editors

#### Data
Only concepts have "editors" filled.

#### Specification

Only entities have audits, i.e. the prop object, the ref object don't. (Could be wrong?)

**Rationale:**  Each entity will have 2 audit creation records. One with the user "import", second with the user "responsible editor.

Implementaions possiblities:
  - make the audits during the parsing : the way
  - make the audits afer the parsing is done (for secondary created entities, I will not have the context, and will be problem to find a responsible editor
  -

In [12]:
tables['R0075_persons']['editor'].value_counts()

Davor Salihović    403
Name: editor, dtype: int64

In [14]:
# the names in the input tables  vs their user ids
# the tuser are imported through the user.json file

#David Zbíral	david.zbiral@mail.muni.cz
#Robert Shaw	robert.shaw@mail.muni.cz
#Jan Král	kral.jan@mail.muni.cz
#Reima Välimäki	reima.valimaki@gmail.com
#Lidia Hinz-Wieczorek	lidhin93@gmail.com
#Davor Salihović	davor.salihovic@mail.muni.cz
#Katia Riccardo	katia.riccardo@mail.muni.cz
#Larissa de Freitas Lyth	larissa.lyth@mail.muni.cz
#Larissa de Freitas Lyth

editors = {"import":"0", "David Zbíral":"100", "Robert Shaw":"101", "Davor Salihović":"102", "Katia Riccardo":"103", "Jan Král":"104", "Reima Välimäki":"105","Lidia Hinz-Wieczorek":"106", "Larissa de Freitas Lyth":"107"}

# Declaration of controlling classes

In [41]:
additional_entities = []
relation_records = []
audits = []

# for controlling entity and mapping of its fields
class EntityMapper:

    # simple inside values mapping from input_values in gsheets to inkVisitor enums
    # field: { FROM  : TO }
    enum_mapper = {'language': {"English":"eng","Latin":"lat","Occitan":"oci","Middle English":"enm","Czech":"ces","Italian":"ita","French":"fra","German":"deu"},"status":{"approved":"1","pending":"0","discouraged":"2","warning":"3"}, "entity_logical_type":{'definite' : "1",
  'indefinite' : "2",   'hypothetical' : "3",  'generic' : "4"}}

    # status  Pending = "0",   Approved = "1",  Discouraged = "2",  Warning = "3",
    valid_entity_classes = ['A','C','E','O','B','R','T','P','G','S','L','NULL']


    IOF = InkVisitorJSONObjectFactory()

    def __init__(self, entity_type, data_row, logger = d.logger):
        self.entity =  type(self).IOF.make(entity_type)
        self.entity_type = entity_type
        self.logger = logger
        self.debug = True
        self.data_row = data_row

    def make_alive(self, legacyId="",label="", label_language="", register_id_where=""):

        self.update_id('make_alive',get_uuid_id())
        self.update_legacyId('make_alive',legacyId)
        self.update_label('make_alive',label)
        self.update_label_language('make_alive',label_language)

        if register_id_where!="" and legacyId!="":
            entity_ids[register_id_where][legacyId] = self.entity['id']

        #save entity to the pool, which goes to import json file
        if pd.DataFrame(additional_entities)[pd.DataFrame(additional_entities)['id']==self.entity['id']].empty:
            additional_entities.append(self.entity)
            #logger.info(f"Appending {legacyId} territory {self.entity['id']} to the pool of additional entities.")
        else:
            pass
            #logger.info(f"Territory {legacyId} {self.entity['id']} exists in the pool of additional entities.")


    def make_prop_object(self,prop_type_id, prop_value_id):
        prop_object = IOF.make('IProp')
        prop_object['id'] = get_uuid_id()
        prop_object['type']['entityId'] = prop_type_id
        prop_object['value']['entityId'] = prop_value_id
        return prop_object

    def prepare_prop_object_data(self,prop_type, prop_value, origin = ""):
        prop_type_id = ""
        prop_value_id = ""

        # logger.info(f"Prepare prop object data>  {prop_type}, {prop_value}")

        # allowed entities in input
        allowed_strict_entities = ['C','M','A','E','L','R','S','V','O','T','P'] # should be followed by numbers
        allowed_free_string_entities = ['~V~']

        # checking input_value
        if not is_valid_uuid(prop_value):
            if any(prop_value.startswith(c)for c in allowed_strict_entities):
                # check for numbers
                if not prop_value[1:4].isnumeric():
                    prop_value = "~V~"+prop_value
            elif not any(prop_value.startswith(c)for c in allowed_free_string_entities):
                prop_value = "~V~"+prop_value

            # logger.info(f"Going to search for id for {prop_value}")
            prop_value_id = self.get_entity_id(prop_value, origin=origin)
        else:
           prop_value_id = prop_value

        # TODO this is useless now, because the line above will make ~V~ from everything unknown
        assert any(prop_value.startswith(c)  for c in allowed_strict_entities) or any(prop_value.startswith(c)  for c in allowed_free_string_entities) or is_valid_uuid(prop_value), f"Prop value unknown, C string or V string entity expected, or valid uuid.{prop_value}, {origin}"

        prop_type_id = self.get_entity_id(prop_type, origin=origin)

        return prop_type_id, prop_value_id



    def make_ref_object(self, ref_resource_id, ref_value_id):
        ref_object = IOF.make('IReference')
        ref_object['id'] = get_uuid_id()
        ref_object['resource'] = ref_resource_id
        ref_object['value'] = ref_value_id
        return ref_object

    def get_entity_id(self,entity_string, origin = ""):
        id = ""
        # logger.info(f"Getting entity string {entity_string} in {origin}")
        try:
            if entity_string.startswith("C") and entity_string[1:4].isnumeric():
                id = entity_ids["concepts"][entity_string]
            elif entity_string.startswith("~V~"):
                ventity = self.make_ventity(entity_string, origin=origin)
                id = ventity['id']
            elif entity_string.startswith("M") and entity_string[1:4].isnumeric():
                id = entity_ids["manuscripts"][entity_string]
            elif entity_string.startswith("A") and entity_string[1:4].isnumeric():
                id = entity_ids["actions"][entity_string]
            elif entity_string.startswith("R") and entity_string[1:4].isnumeric():
                id = entity_ids["resources"][entity_string]
            elif entity_string.startswith("T") and entity_string[1:4].isnumeric():
                id = entity_ids["texts"][entity_string]
            elif entity_string.startswith("O") and entity_string[1:4].isnumeric():
                id = entity_ids["objects"][entity_string]
            elif entity_string.startswith("P") and entity_string[1:4].isnumeric():
                id = entity_ids["persons"][entity_string]
            elif entity_string.startswith("L") and entity_string[1:4].isnumeric():
                id = entity_ids["locations"][entity_string]
            elif entity_string.startswith("E") and entity_string[1:4].isnumeric():
                id = entity_ids["events"][entity_string]
            elif entity_string.startswith("T") and entity_string[1:2].isnumeric():
                id = entity_ids["texts"][entity_string]

            elif is_valid_uuid(entity_string):
                id = entity_string

        except IndexError as E:
            logger.error(f"Cannot get entity id from entity string {entity_string} in {origin}. {E}")
            raise Exception(f"Cannot get entity id '{id}' from value  {entity_string}  in {origin}.")

        if id != "" and isinstance(id, str):
            # logger.info(f"Got entity id {id} from entity string {entity_string} in {origin}")
            return id
        else:
            logger.error(f"Cannot get entity id '{id}' from value {entity_string}  in {origin}.")
            raise Exception(f"Cannot get entity id '{id}' from value  {entity_string}  in {origin}.")

    def make_ventity(self, value_string, origin=""):
        # logger.info(f"Generating ventity from {value_string}.")
        # generate value entity object...
        ventity = IOF.make('IValue')
        ventity['id'] = get_uuid_id()
        ventity['label'] = value_string.replace("~V~","")

        if self.debug:
            ventity['notes'].append(origin)

        # register ventity
        tables['values'] = tables['values'].append({'id':ventity['id'] ,'value':ventity['label'],"origin":origin},ignore_index=True )
        additional_entities.append(ventity)

        # logger.info(f"Ventity id={ventity['id']} generated.")

        # create audit record
        self.create_audit_record(entity_id=ventity['id'], object=ventity)

        return ventity

    def make_rentity(self, label, url = "", origin=""):
        # logger.info(f"Generating rentity from {value_string}.")
        # generate resource entity object...
        rentity = IOF.make('IResource')
        rentity['id'] = get_uuid_id()
        rentity['label'] = label
        rentity['data']['url'] = url

        # if self.debug:
        #    rentity['notes'].append(origin)

        # register rentity
        tables['resources'] = tables['resources'].append({'id':rentity['id'] ,'label':rentity['label'],"dissinet_respository_url":url, "origin":origin}, ignore_index=True )
        additional_entities.append(rentity)

        # logger.info(f"Rentity id={rentity['id']} generated.")
        return rentity


    def make_eentity(self, label, legacyId, url = "", origin=""):

        eentity = IOF.make('IEvent')
        eentity['id'] = get_uuid_id()
        eentity['label'] = label
        eentity['legacyId'] = legacyId

        if self.debug:
            eentity['notes'].append(origin)

        # register event
        tables['events'] = tables['events'].append({'id':eentity['id'] ,'value':eentity['label'],"origin":origin, "legacyId":legacyId}, ignore_index=True )
        additional_entities.append(eentity)

        return eentity

    def make_lentity(self, label, legacyId="", url = "", origin=""):

        lentity = IOF.make('ILocation')
        lentity['id'] = get_uuid_id()
        lentity['label'] = label
        lentity['legacyId'] = legacyId

        if self.debug:
            lentity['notes'].append(origin)

        # register lentity
        tables['locations'] = tables['locations'].append({'id':lentity['id'] ,'value':lentity['label'],"origin":origin,"legacyId":legacyId}, ignore_index=True )
        additional_entities.append(lentity)

        return lentity

    def make_tentity(self, label, legacyId="", url = "", origin=""):

        tentity = IOF.make('ITerritory')
        tentity['id'] = get_uuid_id()
        tentity['label'] = label
        tentity['legacyId'] = legacyId

        if self.debug:
            tentity['notes'].append(origin)

        # register tentity
        tables['territories'] = tables['territories'].append({'id':tentity['id'] ,'value':tentity['label'],"origin":origin,"legacyId":legacyId}, ignore_index=True)
        if legacyId != "":
            entity_ids['texts'][legacyId] = tentity['id']
        additional_entities.append(tentity)

        return tentity

    def make_relation_identity_record(self, id1, id2, certainty="", origin=""):

        rir = IOF.make('Relation.IIdentification')
        rir['id'] = get_uuid_id()
        rir['entityIds'] = [id1,id2]

        if certainty != "":
            rir['certainty'] = certainty

        # register relation
        tables['relations'] = tables['relations'].append({'id':rir['id'] ,'type':rir['type'], "certainty":rir['certainty'],"entityIds":rir['entityIds'], "origin":origin}, ignore_index=True )
        relation_records.append(rir)

        return rir

    def make_relation_classification_record(self, id1, id2, origin=""):

        rcr = IOF.make('Relation.IClassification')
        rcr['id'] = get_uuid_id()
        rcr['entityIds'] = [id1,id2]

        # register relation
        tables['relations'] = tables['relations'].append({'id':rcr['id'] ,'type':rcr['type'], "entityIds":rcr['entityIds'], "origin":origin}, ignore_index=True )
        relation_records.append(rcr)

        return rcr

    def make_relation_record(self, relation_type, id1, id2, origin="", order=1):
        # genereic relation fc
        # BUT synonymic relation have different logic of storage


        # a fork to custom function
        if relation_type == "AAAAAAAAAAAAAAAAAAA":
            rr = self.make_relation_synonym(relation_type, id1, id2, origin="")
        else:
            rr = IOF.make('Relation.I'+relation_type)
            rr['id'] = get_uuid_id()
            rr['entityIds'] = [id1,id2]
            rr['order'] = order

        # logger.info(f"Making {relation_type} record. ID{rr['id']} EIDS{rr['entityIds']}. Origin: {origin}")

        # register lentity
        tables['relations'] = tables['relations'].append({'id':rr['id'] ,'type':rr['type'], "entityIds":rr['entityIds'], "origin":origin}, ignore_index=True )
        relation_records.append(rr)

        return rr

    def make_relation_record_synonym(self, relation_type, id1, id2, origin=""):

        # make new relation object
        rr = IOF.make('Relation.I'+relation_type)
        rr['id'] = get_uuid_id()
        rr['entityIds'] = [id1,id2]

        # check if exists a synonym relation record with id1 or id2
        synonymic_relation_exists = False
        #

        if synonymic_relation_exists:
            pass # will overwrite with the relation object
            # read the relation oject

            # update the relation object

            # delete the relation row in the table

            # delete the relation object in relation_records


        # the calling function will stare the relation record
        return rr

    # interprets prop_type (should be always concept or resource) and input_value (should be entity or value string)
    # get ids of the prop_type and prop_value (possibly creates and register values object)
    # make iProp object
    # puts iProp object into the entity props property
    def hook_prop_object(self, prop_type, input_value, prop_source="",  origin="", prop_source_id ="", field_name="", prop_source_field = ""):

        # allowed entities in type
        assert "C" in prop_type[0] or "R" in prop_type[0], f"Prop type unknown, C or R string entity expected? {prop_type}, {origin}"

        prop_type_id, prop_value_id = self.prepare_prop_object_data(prop_type, input_value, origin=origin)
        # make IProp object
        prop_object = self.make_prop_object(prop_type_id, prop_value_id)

        # register prop object
        tables['props'] = tables['props'].append({'id':prop_object['id'] , 'type_id':prop_type_id,'value_id':prop_value_id,'type':prop_type,'value':input_value, "original_field":field_name, "origin":origin, 'entityId':self.entity['id'], 'legacyId':self.entity['legacyId']}, ignore_index=True)

        if prop_source_field != "":
            self.hook_2ndprop_into_props(prop_object,prop_source_field = prop_source_field, origin = origin)
        elif prop_source_id !="": # means propvalue_2nd
            self.hook_2ndprop_into_props(prop_object,prop_source_id = prop_source_id, origin = origin)
        elif prop_source !="": # means propvalue_2nd
            self.hook_2ndprop_into_props(prop_object,prop_source = prop_source, origin = origin)

        else:
            # hook directly into the entity object
            self.hook_prop_into_props(prop_object)

    def hook_ref_object(self, ref_legacyID, input_value, prop_source="",  origin=""):

        # allowed entities in ref_legacyID
        assert "R" in ref_legacyID, f"Unknown input, R legacyId expected? {ref_legacyID},{input_value}, {origin}"

        #modify value, so the value object is created
        input_value = "~V~"+input_value

        ref_resource_id = self.get_entity_id(ref_legacyID, origin=origin)
        ref_value_id = self.get_entity_id(input_value, origin=origin)

        # make IReference object
        ref_object = self.make_ref_object(ref_resource_id, ref_value_id)

        self.hook_ref_into_refs(ref_object)


    def hook_prop_into_props(self,prop_object):
        self.entity['props'].append(prop_object)

    def hook_ref_into_refs(self,ref_object):
        self.entity['references'].append(ref_object)

    def hook_2ndprop_into_props(self, prop_object, prop_source = "", prop_source_id = "", prop_source_field = "", origin = ""):  # identification by concept id

        # recognition based on the prop_source_field
        if prop_source_field != "":

            # count, value in enumerate(values)
            for count, po in enumerate(self.entity['props']):
                # po['id']
                # find whether this id is registered under the prop_source_field within this entity frame
                candidate_prop_objects = tables['props'][(tables['props']['legacyId'] ==self.entity['legacyId']) & (tables['props']['original_field'] == prop_source_field)]

                #logger.info(f"2nprop: {len(candidate_prop_objects)}")
                #logger.info(f"2nprop: {len(candidate_prop_objects)} : {candidate_prop_objects.iloc[0]['id']}")

                if len(candidate_prop_objects) > 0:
                    for key, row in candidate_prop_objects.iterrows():

                        for  po in self.entity['props']:
                            if po['id'] == row['id'] and len(po['children']) == 0:
                                # logger.info(f"Found prop. {po['id']}. Adding the 2ndprop child.")
                                po['children'].append(prop_object)

                else:
                    logger.error(f"Cannot find the proper prob object record. {prop_object} at {prop_source_field} in origin '{origin}'.")
        else:
            logger.error(f"Fc hook_2ndprop_into_props run without filled prop_source_field in origin '{origin}'.")

        # recognition based on the type concept of the parent prop object
        if prop_source != "":
            keyId = self.get_entity_id(prop_source)
            assert is_valid_uuid(keyId), f"Cannot recognize entity id from {prop_source}"

            # count, value in enumerate(values)
            for count, po in enumerate(self.entity['props']):
                if po['type']['entityId'] == keyId and len(po['children']) == 0:  # I am counting on the fact, that if there are relations from multiples, they are processed in the specific right order
                   po['children'].append(prop_object)


        # recognition based od propobject id
        if prop_source_id != "":
            keyId = prop_source_id
            assert is_valid_uuid(keyId), "Not valid uuid {keyId}"

            for count, po in enumerate(self.entity['props']):
                if po['id'] == keyId:  # I am counting on the fact, that if there are relations from multiples, they are processed in the specific right order
                   po['children'].append(prop_object)



    # method invoker for the INSIDE operation with concrete fields
    def update_inside_field(self, field_name, input_value, origin= ""):
        if input_value != '':

            if ("#" in input_value or "~" in input_value) and field_name!= "note" and "https://docs." not in input_value:
                self.logger.info(f"ALERT # or ~ in the input value {input_value}")

            update_operation = "update_" + field_name
            # update_func = getattr(self, update_operation, self.update_generic)
            update_func = getattr(self, update_operation, self.update_generic)
            update_func(field_name, input_value, origin)
        else:
            raise Exception(f"Trying to update {field_name} with empty input value.")

    #########################################################################################################
    # the naming of procedures corresponds to the name of the input_table fields,  used for inside operations

    def update_label_language(self, field_name="", input_value="", origin = ""):
        if input_value in type(self).enum_mapper['language']:
            self.entity['language'] = type(self).enum_mapper['language'][input_value]
        else:
            self.logger.error(f"Unable to set language in {origin}.")
            self.entity['language'] = input_value # will raise error

    def update_status(self, field_name, input_value, origin = ""):
        if input_value in type(self).enum_mapper['status']:
            self.entity['status'] = type(self).enum_mapper['status'][input_value]
        else:
            self.logger.error(f"Unable to set status in {origin}.")
            self.entity['status'] = input_value # will raise error

    def update_entity_logical_type(self, field_name, input_value, origin = ""):
        if input_value in type(self).enum_mapper['entity_logical_type']:
            self.entity['data']['logicalType'] = type(self).enum_mapper['entity_logical_type'][input_value]
        else:
            self.logger.error(f"Unable to set entity logical type in {origin}.")
            self.entity['data']['logicalType'] = input_value # will raise error

    def update_note(self, field_name, input_value, origin = ""):
        #self.logger.info(f"Updating note with {input_value}.")
        if "#" in input_value:
            values = [v.strip() for v in input_value.split("#")]
            for v in values:
                self.entity['notes'].append(v)
        else:
            self.entity['notes'].append(input_value)

    def update_id(self, field_name, input_value, origin = ""):
        # self.entity['id'] = input_value
        self.entity['id'] = input_value

    def update_legacyId(self, field_name="", input_value="", origin = ""):
        # logger.info(f"Trying to set legacyId {type(input_value)}:'{input_value}' {origin}.")
        self.entity['legacyId'] = input_value

    def update_label(self, field_name="", input_value="", origin = ""):
        self.entity['label'] = input_value

    def update_wordnet_lemma_id(self, field_name, input_value, origin = ""):
        # self.logger.info(f" wordnet_lemma_id NOT IMPLEMENTED ")
        pass

    def update_wordnet_synset_id(self, field_name, input_value, origin = ""):
        # self.logger.info(f" wordnet_synset_id NOT IMPLEMENTED ")
        pass

    def update_generic(self, field_name, input_value, origin = ""):
        self.entity[field_name] = input_value


    def create_audit_record(self, entity_id = '', editor_candidate = '', object = {}):

        data_row = self.data_row
        if entity_id == '':
            entity_id = self.entity['id']
        if object == {}:
            object = self.entity  # should be run at the end of the entity making, to the object is "full"

        # date =  {
        #     "$reql_type$": "TIME",
        #      "epoch_time": time.time(),
        #      "timezone": "+00:00"
        # }
        date =  datetime.now().strftime("%Y-%m-%dT%H:%M:%S")+"Z" #2021-12-05T19:10:15.739Z

        if editor_candidate == '' and 'editor' in data_row.keys():
            editor_candidate = data_row['editor']
        if editor_candidate in editors.keys():
            editor_id = editors[editor_candidate]
        else:
            editor_id = editors['David Zbíral']

        import_audit = IOF.make('IAudit')
        import_audit['id'] = get_uuid_id()
        import_audit['entityId'] = entity_id
        import_audit['user'] =  editors['import']
        import_audit['date'] = date
        import_audit['changes'] = object

        editor_audit = IOF.make('IAudit')
        editor_audit['id'] = get_uuid_id()
        editor_audit['entityId'] = entity_id
        editor_audit['user'] =  editor_id
        editor_audit['date'] = date
        editor_audit['changes'] = object

        # audits.append(import_audit)
        audits.append(editor_audit)


class TerritoryEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

        if "legacyId" in data_row and "T" in data_row["legacyId"]:
            self.entity['data']['parent']['order'] = int(data_row["legacyId"].replace("T",""))
        else:
            # logger.info(f"In territory entity mapper, legacyId uknown, the order of the entity must be set manually.")
            pass

class ConceptEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class ActionEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger,):
        EntityMapper.__init__(self,entity_type, data_row, logger)

    def update_subject_entity_type(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: subject_entity_type")
        entities = [e.strip() for e in value.split("|")]
        for e in entities:
            if e in self.valid_entity_classes:
                self.entity['data']['entities']['s'].append(e)
            elif e == "*":
                self.entity['data']['entities']['s'] = self.valid_entity_classes
            else:
                logger.error(f"Non valid entity processed: {e} while AP.update_subject_entity_type().")

    def update_subject_valency(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: subject_valency")
        self.entity['data']['valencies']['s'] = value

    def update_actant1_entity_type(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant1_entity_type")
        entities = [e.strip() for e in value.split("|")]
        for e in entities:
            if e in self.valid_entity_classes:
                self.entity['data']['entities']['a1'].append(e)
            elif e == "*":
                self.entity['data']['entities']['a1'] = self.valid_entity_classes
            else:
                logger.error(f"Non valid entity processed: {e} while AP.update_actant1_entity_type().")

    def update_actant2_entity_type(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant2_entity_type")
        entities = [e.strip() for e in value.split("|")]
        for e in entities:
            if e in self.valid_entity_classes:
                self.entity['data']['entities']['a2'].append(e)
            elif e == "*":
                self.entity['data']['entities']['a2'] = self.valid_entity_classes
            else:
                logger.error(f"Non valid entity processed: {e} while AP.update_actant2_entity_type().")

    def update_actant1_valency(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant1_valency")
        self.entity['data']['valencies']['a1'] = value

    def update_actant2_valency(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant1_valency")
        self.entity['data']['valencies']['a2'] = value


class ResourceEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

    def update_url(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: subject_entity_type")
        self.entity['data']['url'] = value


class ObjectEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class EventEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class LocationEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class PersonEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)


class EntityMapperFactory:
    def __init__(self):
        pass

    def make(self, entity_name, data_row):
        if 'ITerritory' == entity_name:
            return TerritoryEntityMapper(entity_name, data_row)
        elif 'IAction' == entity_name:
            return ActionEntityMapper(entity_name, data_row)
        elif 'IConcept' == entity_name:
            return ConceptEntityMapper(entity_name,data_row)
        elif 'IResource' == entity_name:
            return ResourceEntityMapper(entity_name, data_row)
        elif 'IObject' == entity_name:
            return ObjectEntityMapper(entity_name, data_row)
        elif 'IEvent' == entity_name:
            return EventEntityMapper(entity_name, data_row)
        elif 'ILocation' == entity_name:
            return LocationEntityMapper(entity_name, data_row)
        elif 'IPerson' == entity_name:
            return LocationEntityMapper(entity_name, data_row)

        else:
            logger.warning(f"Unrecognized entity class in entity mapper. Is this right?")
            return EntityMapper(entity_name)





# CONTROL CLASS
class ParseController():

    def __init__(self, entity_list = [], existing_parser = False, reparse_entity_list = [],  keyword_row_id = 3,  logger = d.logger):
        self.entity_list = entity_list
        self.logger = logger
        self.parsers = {}
        self.js_objects = []
        self.existing_parser = existing_parser
        self.reparse_entity_list = reparse_entity_list

        if existing_parser and isinstance(existing_parser, ParseController):

            self.parsers = existing_parser.parsers
            self.logger = existing_parser.logger
            self.js_objects = existing_parser.js_objects

            for e in self.entity_list:
                if 'texts' in e and 'texts' in self.reparse_entity_list:
                    self.parsers[e] = TextParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'actions' in e and 'actions' in self.reparse_entity_list:
                    self.parsers[e] = ActionParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'concepts' in e and 'concepts' in self.reparse_entity_list:
                    self.parsers[e] = ConceptParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'resources' in e and 'resources' in self.reparse_entity_list:
                    self.parsers[e] = ResourceParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'manuscripts' in e and 'manuscripts' in self.reparse_entity_list:
                    self.parsers[e] = ManuscriptParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'R0006_persons' in e and 'R0006_persons' in self.reparse_entity_list:
                    self.parsers[e] = PersonParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'R0007_locations' in e and 'R0007_locations' in self.reparse_entity_list:
                    self.parsers[e] = LocationParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'R0008_events' in e and 'R0008_events' in self.reparse_entity_list:
                    self.parsers[e] = EventParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'R0075_persons' in e and 'R0075_persons' in self.reparse_entity_list:
                    self.parsers[e] = PersonParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'R0035_locations' in e and 'R0035_locations' in self.reparse_entity_list:
                    self.parsers[e] = LocationParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'R0083_events' in e and 'R0083_events' in self.reparse_entity_list:
                    self.parsers[e] = EventParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                else:
                    self.logger.warning(f"Coming to basic Parser entity - strange '{e}' {type(e)}.")
                    self.parsers[e] = Parser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)

        else:
            for e in self.entity_list:
                if 'texts' in e:
                    self.parsers[e] = TextParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'actions' in e:
                    self.parsers[e] = ActionParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'concepts' in e:
                    self.parsers[e] = ConceptParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'resources' in e:
                    self.parsers[e] = ResourceParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'manuscripts' in e:
                    self.parsers[e] = ManuscriptParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'R0006_persons' in e:
                    self.parsers[e] = PersonParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'R0007_locations' in e:
                    self.parsers[e] = LocationParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'R0008_events' in e:
                    self.parsers[e] = EventParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'R0075_persons' in e:
                    self.parsers[e] = PersonParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'R0035_locations' in e:
                    self.parsers[e] = LocationParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                elif 'R0083_events' in e:
                    self.parsers[e] = EventParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
                else:
                    self.logger.warning(f"Coming to basic Parser entity - strange '{e}' {type(e)}.")
                    self.parsers[e] = Parser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)

    def load_json_objects(self):
        for name, p in self.parsers.items():
            self.js_objects = self.js_objects + p.js_objects

    def parse(self, stop = False):
        for name, p in self.parsers.items():
            if len(self.reparse_entity_list) > 0:
                if name in self.reparse_entity_list:
                    self.logger.info("PARTIAL PARSE Active, reparsing "+name)
                    p.parse_rows(stop)
                else:
                    self.logger.info("PARTIAL PARSE Active, skipping "+name)
            else:
                p.parse_rows(stop)


# WORKER CLASS
class Parser():
    EMP = EntityMapperFactory()

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        self.name = name
        self.logname = name.upper()
        self.input_header_df = header_df
        self.input_table_df = table_df
        self.prepared_table = pd.DataFrame()
        self.keyword_row_id =  keyword_row_id
        self.columns = self.input_header_df.columns.tolist()

        self.proptype_2nd = {} # for registering the columns, which contains proptype_2nd information

        self.parsing_instruction = {}
        self.oper_columns = {'discard':[],'inside':[],'special':[],'unknown':[],"proptype":[],'propvalue':[],'proptype_2nd':[],'propvalue_2nd':[],"dependent":[], "reference":[], "reference_object":[], "reference_part":[],"hooked-inside":[],"hooked-propvalue":[],"hooked-relation":[]}
        self.logger = logger

        # parsed json data holder
        self.js_objects = []

        # RUN
        self.process_header_instructions()
        self.prepare_input_table()

    # "parsing" instructions
    def process_header_instructions(self) -> (pd.DataFrame, pd.DataFrame):
        keyword_row = self.input_header_df.iloc[self.keyword_row_id]
        prop_type_row = self.input_header_df.iloc[self.keyword_row_id - 1]
        source_node_row = self.input_header_df.iloc[self.keyword_row_id - 2]

        log_uncertain_instructions = []

        for c in self.columns:
            instruction_candidate = str(keyword_row.at[c]).strip()
            prop_type_candidate = str(prop_type_row.at[c]).strip()
            source_node_candidate = str(source_node_row.at[c]).strip()

            if c == '':
                self.logger.error(f"{self.logname} There is empty column in the dataset.")
                raise Exception(f"{self.logname} There is empty column in the dataset.")

            if "?" in instruction_candidate or "?" in prop_type_candidate or "?" in source_node_candidate:
                log_uncertain_instructions.append(f"{c.upper()}:{instruction_candidate},{prop_type_candidate},{source_node_candidate}")
                instruction  = {'operation':'discard', 'target': None}
                self.oper_columns['discard'].append(c)


            # known instructions
            if 'discard' in instruction_candidate:
                instruction  = {'operation':'discard', 'target': None}
                self.oper_columns['discard'].append(c)

            elif instruction_candidate == "propvalue":
                prop_type = prop_type_candidate
                source_node = source_node_candidate

                if source_node != "" and prop_type == "":
                    prop_type = source_node

                # test whether is its propvalue proper or dependent (=proptype is dynamic, value from another column)
                if prop_type.strip() == "":
                    self.oper_columns['dependent'].append(c)
                    logger.info(f"Throwing out propvalue instruction in column {c}.")
                    continue # ignoring "dependent propvalue"

                if "?" in prop_type or "?"  in source_node:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'propvalue', 'type': prop_type, 'source':source_node} # source can be ignored, because the iProp object is sitting inside of it
                    self.oper_columns['propvalue'].append(c)

            elif 'propvalue_2nd' in instruction_candidate:
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                if "?" in prop_type or "?"  in source_node:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'propvalue_2nd', 'type': prop_type, 'source':source_node} # source can NOT be ignored, it signals which existing iProp object will hold this iProp object
                    self.oper_columns['propvalue_2nd'].append(c)

            elif 'proptype_2nd' in instruction_candidate:
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                if "?" in prop_type or "?"  in source_node:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'proptype_2nd', 'type': prop_type, 'source':source_node}
                    self.oper_columns['proptype_2nd'].append(c)

            elif 'special' == instruction_candidate:
                # looks for custom functions registered by column name
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                instruction  = {'operation':'special', 'type': prop_type, 'source':source_node}
                self.oper_columns['special'].append(c)

            elif 'proptype' in instruction_candidate:
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                instruction  = {'operation':'proptype', 'type': prop_type, 'source':source_node}
                #logger.info(f"here ...{instruction_candidate} {c}")
                self.oper_columns['proptype'].append(c)

            elif 'dependent' in instruction_candidate:
                # ignore
                # the value is solved by another instruction
                instruction  = {'operation':'dependent', 'type': prop_type, 'source':source_node}

            elif instruction_candidate == "inside":
                if "?" in c:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'inside', 'target': None}
                    if len(prop_type_candidate) > 0:
                        instruction  = {'operation':'inside', 'target': prop_type_candidate}

                    self.oper_columns['inside'].append(c)

            elif instruction_candidate == "hooked-inside":
                if "?" in c:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'hooked-inside', 'target': None}
                    if len(prop_type_candidate) > 0:
                        instruction  = {'operation':'hooked-inside', 'target': prop_type_candidate}

                    self.oper_columns['hooked-inside'].append(c)

            elif instruction_candidate == "hooked-propvalue":
                if "?" in c:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'hooked-propvalue', 'target': None}
                    if len(prop_type_candidate) > 0:
                        instruction  = {'operation':'hooked-propvalue', 'target': prop_type_candidate}

                    self.oper_columns['hooked-propvalue'].append(c)

            elif instruction_candidate == "hooked-relation":
                if "?" in c:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'hooked-relation', 'target': None}
                    if len(prop_type_candidate) > 0:
                        instruction  = {'operation':'hooked-relation', 'target': prop_type_candidate}

                    self.oper_columns['hooked-relation'].append(c)

            elif "reference" in instruction_candidate:
                #logger.info(f"Found instruction 'reference'. {prop_type_candidate}")
                self.oper_columns["reference"].append(c)
                instruction  = {'operation':'reference', 'ref_legacy_id':prop_type_candidate}

            elif "reference_object" in instruction_candidate:
                #logger.info(f"Found instruction 'reference'. {prop_type_candidate}")
                self.oper_columns["reference_object"].append(c)
                instruction  = {'operation':'reference_object', 'ref_legacy_id':prop_type_candidate}

            elif "reference_part" in instruction_candidate:
                #logger.info(f"Found instruction 'reference'. {prop_type_candidate}")
                self.oper_columns["reference_part"].append(c)
                instruction  = {'operation':'reference_part', 'ref_legacy_id':prop_type_candidate}

            elif instruction_candidate == "relation":
                relation_type = prop_type_candidate
                instruction  = {'operation':'relation', 'type': relation_type}

            else:
                instruction = {'operation':'unknown', 'target': None}
                self.oper_columns['unknown'].append(c)
            self.parsing_instruction[c] = instruction

        self.logger.info(f"{self.logname} Uncertain parsing instructions in {len(log_uncertain_instructions)} columns: " + " ".join(log_uncertain_instructions) + ".")
        return self.parsing_instruction

    def prepare_input_table(self):
        ip = self.input_table_df.copy()

        # discard  columns with discard and unknown operations
        for c in self.oper_columns['discard']+self.oper_columns['unknown']:
            ip.drop(columns=c, inplace=True)
        #ip.drop(columns=self.oper_columns['discard']+self.oper_columns['unknown'], inplace=True)

        self.logger.info(f"{self.logname} {len(self.oper_columns['discard']+self.oper_columns['unknown'])} columns have been dropped (discard:{len(self.oper_columns['discard'])}, unknown:{len(self.oper_columns['unknown'])}: {self.oper_columns['unknown']}). Table now has {len(ip.columns)} columns, inside:{len(self.oper_columns['inside'])},propvalue:{len(self.oper_columns['propvalue'])}, special:{len(self.oper_columns['special'])}, proptype: {len(self.oper_columns['proptype'])}, proptype_2nd: {len(self.oper_columns['proptype_2nd'])}, dependent:{len(self.oper_columns['dependent'])}, reference_part:{len(self.oper_columns['reference_part'])}. Originally {self.input_table_df.shape[1]} columns.")

        self.prepared_table = ip

    def prepare_property(self):
        pass

    def make_row_object(self, data_row):
        class_name = table_to_entity[self.name]
        return self.EMP.make(class_name, data_row)

    def itemize_valuestring_for_multiples(self, value_with_multiples, origin="") -> []:
        values = []
        if isinstance(value_with_multiples,str):
            parsed_value = value_with_multiples.split('#')
            values =  [item.strip() for item in parsed_value]
        else:
            raise Exception(f"Expected value to be string. Got {type(value_with_multiples)}. {origin}")
        return values

    def parse_rows(self, stop = False):
        self.logger.info(f"Starting to parse {self.name}.")

        if  self.prepared_table['label'].isnull().any():
            self.prepared_table  = self.prepared_table[self.prepared_table['label'].notna()]
            self.logger.info(f"Empty labels found in {self.name} table. (new entities added through parsing process). Adjusting - entities with empty labels not taken into account.")

        counter = 0
        for key, row in self.prepared_table.iterrows():
            counter += 1
            if stop and counter > stop:
                break
            #self.logger.info(f"{self.name} Processing row {key}")
            #self.logger.info(f"{row.to_dict()}")
            entity_mapper = self.make_row_object(row.to_dict())

            if row['legacyId'] == "":
                logger.info(f"Skipping {row['label']}, does not have set legacyId. Remnant of already run parse?")
                continue

            for name, value in row.items():
                if name in self.parsing_instruction:
                    operation = self.parsing_instruction[name]
                else:
                    continue # silently ignore unknown columns
                # logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")

                # force string
                value = str(value)

                # validation of value for question marks
                if "??" in value:
                    logger.info(f"About ??? : There is {value} in column {name} in row {str(key)}.")
                    continue

                # thrashing "NA"
                #if value == "NA":
                #    continue

                if operation['operation'] == 'inside' and value != '' and '?' not in name:
                    # logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")
                    if operation['target']:
                        name = operation['target']
                    entity_mapper.update_inside_field(name,value,operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value))

                if operation['operation'] == 'propvalue' and value != '':

                    #logger.info(f"{self.name} Processing 'propvalue' for column {name}, with value {value} and type {operation['type']} Op:{operation}")
                    prop_type = operation['type']

                    if prop_type in row.keys():  # prop type is defined in other column (it is NOT fixed for all propvalues)
                        prop_type = row[prop_type]
                        # logger.info(f"Shifting to remote proptype {prop_type} in the context of {name}, {value},{operation['type']}.")

                    if prop_type == '' or 'C' not in prop_type:
                        raise Exception(f"Propvalue does not have prop type '{prop_type}' defined. C entity-string expected, got {key}, {name}, {value}. [{operation}]")

                    for item in self.itemize_valuestring_for_multiples(value):
                        #logger.info(f"Propvalue value {value} item {item}.")
                        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = item, origin = operation['operation'] +">"+ str(key)+f":{row['legacyId']}"+":"+str(name)+":"+str(value), field_name = name)


                if operation['operation'] == 'proptype_2nd' and value != '':
                    # logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")

                    #register proptype_2nd for the source
                    self.proptype_2nd[operation['source']] = name


                    # prop_type = operation['type']
                    # if prop_type == '' or 'C' not in prop_type:
                    #     raise Exception(f"Propvalue does not have prop type defined. C entity-string expected, got {key}, {name}, {value}")
                    # for item in self.itemize_valuestring_for_multiples(value):
                    #     entity_mapper.hook_prop_object(prop_type = prop_type, input_value = item, origin = operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value), field_name = name)

                if operation['operation'] == 'propvalue_2nd' and value != '':
                    #logger.info(f"{self.name} propvalue_2nd : Processing columns {name}, with value {value}. Op:{operation}")
                    prop_type = operation['type']
                    prop_source_name = operation['source']  # header_name,  we need concept_id
                    prop_source_id = self.parsing_instruction[prop_source_name]['type']

                    #logger.info(f"OP provalue_2nd with prop_source_name: '{prop_source_name}' : prop_source_id '{prop_source_id}' : value '{value}'")

                    #logger.info(f"There are registered proptype_2nd: {self.proptype_2nd}")
                    # logger.info(f"There are parsing instructions : {self.parsing_instruction}")

                    # assert 'C' in prop_source_id, f"Trying to get to the concept of 1st level property to address 2nd level property. Context {self.name} propvalue_2nd : Processing columns {name}, with value {value}. Op:{operation} in {row}."

                    # hack for Castellario events? other datasets do not need it ? why????
                    #if prop_type == '' and prop_source_id == '':
                    #    prop_type = row[prop_source_name]

                    # there can be two situations
                    # B - proptype is defined in proptype_2nd column
                    if prop_source_name in self.proptype_2nd.keys():
                        # logger.info(f"We have registered source type column {self.proptype_2nd[prop_source_name]} for this propvalue_2nd.")
                        prop_type = row[self.proptype_2nd[prop_source_name]]
                    else:
                        # A - proptype is fixed for the whole column, defined in proptype_2nd
                        if prop_type == '' or 'C' not in prop_type:
                            raise Exception(f"Propvalue_2nd does not have prop type defined. C entity-string expected, got prop_type '{prop_type}', key:{key}, name:{name}, value:{value}")

                    origin = operation['operation'] +">"+ str(key)+f":{row['legacyId']}"+":"+str(name)+":"+str(value)

                    for item in self.itemize_valuestring_for_multiples(value, origin=origin):
                        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = item, prop_source = prop_source_id, origin = origin, field_name = name, prop_source_field = prop_source_name)

                if operation['operation'] == 'special' and value != "":
                    # logger.info(f"SPECIAL {operation} {value}")
                    func = getattr(self, 'special_'+name)
                    func(operation, value, entity_mapper)

                if operation['operation'] == 'reference' and value != "":
                    pass
                if operation['operation'] == 'reference_object' and value != "":
                    pass

                if operation['operation'] == 'reference_part' and value != "":
                    ref_object_legacy_id = operation['ref_legacy_id']

                    if ref_object_legacy_id in row.keys():
                        ref_object_legacy_id = row[ref_object_legacy_id]

                    logger.info(f"Doing {operation}, {ref_object_legacy_id}, '{value}'. Row {row}.")
                    entity_mapper.hook_ref_object(ref_legacyID = ref_object_legacy_id, input_value = value, origin = operation['operation'] +">"+ ":"+ name + " " +str(value))

                if operation['operation'] == 'relation' and value != "" and value != "NA":
                    relation_type = operation['type']
                    related_object = value

                    id1 = entity_mapper.entity['id']

                    #for order,item in enumerate(reversed(self.itemize_valuestring_for_multiples(related_object)),1):
                    for order,item in enumerate(self.itemize_valuestring_for_multiples(related_object),1):
                        if item[0] not in entity_mapper.valid_entity_classes:
                            logger.error(f"The string {item} does not refer to valid entity class - trying to instantiate relation {relation_type} in row {key}. ")
                            continue
                        id2 = entity_mapper.get_entity_id(item, origin = str(key)+ " Getting second relation entity. "+relation_type+ " "+item )
                        entity_mapper.make_relation_record(relation_type, id1, id2, origin=operation['operation'] +">"+ ":"+ relation_type + f" {entity_mapper.entity['legacyId']}>{item} " +str(id1)+" "+str(id2), order=order*10)

            self.js_objects.append(entity_mapper.entity)

            # make audit records
            entity_mapper.create_audit_record()

            # break  DEV, checking parsing after first iteration


    def special_editor(self, operation, value, entity_mapper, field_name="special_editor"):
        pass


class TextParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    # special methods for fields, which needs fully individual processing
    def special_edition_1(self, operation, value, entity_mapper, field_name="edition_1", ):
        # Parse this col. as "propvalue" - but you need to generate the target entities since they do not exist. How to do it: for any value, create an R entity with "label" = textual value in this col., "label language" = English, "status" = "approved", and "URL" = the hyperlink in the formula sitting on the textual value in this col. As usual, ignore NS and NA values (exact match) - do not import anything if the value is NA.
        # logger.info(f"Special edition1 running ...{operation} {value}")

        origin = operation['operation'] +" "+ operation['type']+ ">"+ ":"+field_name + str(value)
        prop_type = operation['type']

        # make rentity
        # logger.info(f"Rentity making {value}")
        if "|" in value:
            data = value.split("|")
            label = data[0]
            url = data[1]
        else:
            url = ""
            label = value
            # logger.warning(f"Expected char | signaling url after label. Got just {value}."+origin)

        # generate resource entity if it does not exist
        # check by label and url
        tdf = tables['resources'].fillna("")
        check_rentity = tdf[(tdf['label'] == label) & (tdf['dissinet_repository_url'] == url)]
        #check_rentity = tdf[(tdf['label'] == label)  & (~tdf['dissinet_repository_url'].isna() | ~(tdf['dissinet_repository_url']==""))]
        if check_rentity.empty:
            rentity = entity_mapper.make_rentity(label, url, origin=origin)
            rentity['language'] = entity_mapper.enum_mapper['language']['English']
        else:
            rentity = {'id': check_rentity.iloc[0]['id']}

        entity_mapper.create_audit_record(entity_id=rentity['id'], object=rentity)
        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = rentity['id'], origin = origin, field_name=field_name)

    def special_edition_2(self, operation, value, entity_mapper):
        self.special_edition_1( operation, value, entity_mapper, field_name="edition_2")

    def special_edition_3(self, operation, value, entity_mapper):
        self.special_edition_1( operation, value, entity_mapper, field_name="edition_3")

    def special_creation_event_id(self, operation, value, entity_mapper : EntityMapper, field_name="creation_event_id"):
        # old instructions
        # Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new “hash” ID from the db, (3) label of this E: see next col., (4) logical type “definite” (default), (5) label language “English”, (6) status “approved”, and (7) attach to any of those Es the metaprop "(has) - C0565 “class” - C2642 “creation” (to instantiate the event to its event type = event class).

        # new instructionds
        # DONE
        # Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: see next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to this E the Relation of the type Classification leading to C2642 "creation".

        data = entity_mapper.data_row
        origin = operation['operation']+ " " + data['legacyId'] + ">"+ ":"+field_name +" "+ str(value)

        event_entity = entity_mapper.make_eentity(data['creation_event_label'], legacyId = value, origin = origin)
        event_entity['language'] = entity_mapper.enum_mapper['language']['English']

        # prop_type_id = entity_mapper.get_entity_id("C0565")
        # prop_value_id = entity_mapper.get_entity_id("C2642")
        # make IProp object
        # prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
        # hook prop object to the event entity
        # event_entity['props'].append(prop_object)
        # hook the vent event to the territory
        # entity_mapper.hook_prop_object(prop_type = "C2642", input_value = event_entity['id'], origin = origin, field_name=field_name)

        # make event part of territories props
        entity_mapper.hook_prop_object(prop_type = "C2642", input_value = event_entity['id'], origin = origin, field_name=field_name)
        # (7) attach to this E the Relation of the type Classification leading to C2642 "creation"
        entity_mapper.make_relation_record('Classification',event_entity['id'],entity_mapper.get_entity_id("C2642"))

        # process time_relations
        t_relations = [(
            data["timerelation1_type_conceptified_id"], data["timerelation1_target_id"]
        ), (
            data["timerelation2_type_conceptified_id"], data["timerelation2_target_id"]
        ),(
            data["timerelation3_type_conceptified_id"], data["timerelation3_target_id"]
        ), (
            data["timerelation4_type_conceptified_id"], data["timerelation4_target_id"]
        )]

        for o in t_relations:
            if len(o[0]) > 0 and len(o[1]) > 0:
                prop_type_id = entity_mapper.get_entity_id(o[0], origin = origin)
                prop_value_id = entity_mapper.get_entity_id("~V~"+o[1], origin = origin)
                prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
                event_entity['props'].append(prop_object)

        # create audit record
        entity_mapper.create_audit_record(entity_id=event_entity['id'], object=event_entity)


    # empty, operation is solved by f above
    def special_creation_event_label(self, operation, value, entity_mapper):
        pass


class ActionParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)


class ConceptParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    def special_wordnet_synset_id(self, operation, value, entity_mapper,field_name="special_wordnet_synset_id",):
        # wordnet_resource_id = R0067
        entity_mapper.hook_ref_object(ref_legacyID = "R0067", input_value = value, origin = operation['operation'] +">"+ ":"+field_name + " " +str(value))


class ManuscriptParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    def special_creation_event_id(self, operation, value, entity_mapper : EntityMapper, field_name="creation_event_id"):
        # Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new “hash” ID from the db, (3) label of this E: see next col., (4) logical type “definite” (default), (5) label language “English”, (6) status “approved”, and (7) attach to any of those Es the metaprop "(has) - C0565 “class” - C2642 “creation” (to instantiate the event to its event type = event class).

        # new instructions
        # TODO
        # Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: take from next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to this E the Relation of the type Classification leading to C2642 "creation".

        origin = operation['operation'] +">"+ ":"+field_name +" "+ str(value)

        data = entity_mapper.data_row
        event_entity = entity_mapper.make_eentity(data['creation_event_label'], legacyId = value, origin = origin)
        event_entity['language'] = entity_mapper.enum_mapper['language']['English']

        # OLD way, we have relations now
        #prop_type_id = entity_mapper.get_entity_id("C0565")
        #prop_value_id = entity_mapper.get_entity_id("C2642")
        # make IProp object
        #prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
        # hook prop object to the event entity
        #event_entity['props'].append(prop_object)

        # hook the vent event to the territory
        entity_mapper.hook_prop_object(prop_type = "C2642", input_value = event_entity['id'], origin = operation['operation'] +">"+ ":"+field_name + str(value))

        # (7) attach to this E the Relation of the type Classification leading to C2642 "creation"
        entity_mapper.make_relation_record('Classification',event_entity['id'],entity_mapper.get_entity_id("C2642"))

        # process time_relations
        t_relations = [(
            data["timerelation1_type_conceptified_id"], data["timerelation1_target_id"]
        ), (
            data["timerelation2_type_conceptified_id"], data["timerelation2_target_id"]
        ),(
            data["timerelation3_type_conceptified_id"], data["timerelation3_target_id"]
        ), (
            data["timerelation4_type_conceptified_id"], data["timerelation4_target_id"]
        )]

        origin = origin + "LegacyId: "+ data['legacyId']
        for o in t_relations:
            if len(o[0]) > 0 and len(o[1]) > 0:
                prop_type_id = entity_mapper.get_entity_id(o[0], origin = origin)
                prop_value_id = entity_mapper.get_entity_id("~V~"+o[1], origin = origin)
                prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
                event_entity['props'].append(prop_object)

        # create audit record
        entity_mapper.create_audit_record(entity_id=event_entity['id'], object=event_entity)


    # empty, operation is solved by f above
    def special_creation_event_label(self, operation, value, entity_mapper):
        pass


    def special_repository_label(self, operation, value, entity_mapper, field_name = "repository_label"):
        # For each non-empty, non-NA, non-NS row: (1) generate L entity with label = value in this col., status = “approved”, entity logical type = “definite”, label language = value in the next col. (repository_label_language); (2) append to this L entity a metaprop (has) - C0565 “class” - C2646 “archive or library”, and (3) under the O entity representing the row (i.e. the physical manuscript), add a metaprop which will relate this O to this L entity (repository) through the relation: O - (has) - C2645 “repository” - L in this col.

        origin = f"Making location '{value}' from " + entity_mapper.data_row['legacyId'] + f" by field {field_name}."

        if value != "" and value!="NA" and value!="N/A" and value!="NS":  # check value
            # check whether location exists with the same label
            if tables["locations"][tables["locations"]['value']==value].empty: # NO
                lentity = entity_mapper.make_lentity(label=value, legacyId="L_from_"+entity_mapper.data_row['legacyId'], origin=origin)
                lentity['language'] = entity_mapper.enum_mapper['language'][entity_mapper.data_row['repository_label_language']]
            else: #YES
                lentity = {}
                lentity['id'] = tables["locations"][tables["locations"]['value']==value].iloc[0]['id']

            # OLD metaprop way
            #prop_type_id = entity_mapper.get_entity_id("C0565", origin = origin)
            #prop_value_id = entity_mapper.get_entity_id("C2646", origin = origin)
            #prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
            #lentity['props'].append(prop_object)

            #(2) append to this L entity a RelationType.Classification leading to C2646 "archive or library"
            entity_mapper.make_relation_classification_record(lentity['id'], entity_mapper.get_entity_id('C2646'))

            # create audit record
            entity_mapper.create_audit_record(entity_id=lentity['id'], object=lentity)

            # let the manuscript O own the location
            entity_mapper.hook_prop_object(prop_type = "C2645", input_value = lentity['id'], origin = origin)


    # solves the method above
    def special_repository_label_language(self, operation, value, entity_mapper):
        pass

    def special_reproduction_online_url(self, operation, value, entity_mapper, field_name = "reproduction_online_url"):
        # If non-empty, non-NA, (1) generate an R entity with label "Reproduction of " + label of the MS (i.e. value in the B column, status = “approved”, label-language = “English”, url = the URL sitting under the hyperlink value in this cell, and (2) add metaprop to the O entity represented by this row: O - (has) - C1199 “digital reproduction” - the R entity here generated.

        origin = f"Making Resource entity '{value}' from " + entity_mapper.data_row['legacyId'] + f" by field {field_name}."

        if value != "" and value!="NA" and value!="N/A" and value!="NS":  # check value

            if "|" in value:
                data = value.split("|")
                label = data[0]
                url = data[1]

                if label == "link":
                    label = "Reproduction of " + entity_mapper.data_row['label']

            else:
                url = ""
                label = value
                logger.warning(f"Expected char | signaling url after label. Got just {value}. "+origin)


            tdf = tables['resources']
            check_rentity = tdf[(tdf['label'] == label) & (tdf['dissinet_repository_url'] == url)]
            if check_rentity.empty:
                rentity = entity_mapper.make_rentity(label, url, origin=origin)
                rentity['language'] = entity_mapper.enum_mapper['language']['English']
            else:
                rentity = {'id': check_rentity.iloc[0]['id']}

            entity_mapper.hook_prop_object(prop_type = "C1199", input_value = rentity['id'], origin = origin)

            # create audit record
            entity_mapper.create_audit_record(entity_id=rentity['id'], object=rentity)


class ResourceParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)


class PersonParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    # TODO implement
    def special_introducers_id(self, operation, value, entity_mapper, field_name = "introducers_id"):
        pass


class EventParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    def checkTerritoryExists(self,legacyId):
        if "-" in legacyId:
            if pd.DataFrame(additional_entities)[pd.DataFrame(additional_entities)['legacyId']== legacyId].empty:
                return False
            else:
                return True
        else: # main text
            if tables["texts"][tables["texts"]["legacyId"]==legacyId].empty:
                return False
            else:
                return True

    def special_subterritory_id(self, operation, value, entity_mapper, field_name = "subterritory_id"):
        # Light yellow: document aka Territory metadata (not Event).
        # please generate these T entities following Adam's script for parsing Robert's Sellan coding sheet IDs into a hierarchical T structure
        # and put them under their proper root T through legacy IDs (first element of ID = root T ID).
        # E.g. here, in Guglielmites, there is root, i.e. the whole T; then parts 1-4; and underneath them, the individual documents.
        # Entity type = T; T label and label_language defined in the two respective cols.; entity_logical_type = definite; status = approved. Other metadata set as metaprops in the following yellow cols.

        datarow = entity_mapper.data_row.copy()

        # override of legacyId
        # datarow['legacyId'] = value

        # create territory proper
        trt = self.EMP.make(entity_name="ITerritory", data_row=datarow)
        trt.make_alive(legacyId= datarow["subterritory_id"], label=datarow['document_label'], label_language=datarow['document_label_language'], register_id_where="texts")

        trt.create_audit_record(entity_id=trt.entity['id'], object=trt.entity)

        # propvalues
        prop_value_fields = {"state_of_conservation_id":"C3440", "participant_id":"C2863", "inquisitor_id":"C1541", "notary_id":"C1652", "witness_assessor_id":"C1540"}

        for pvf, concept in prop_value_fields.items():
            if pvf not in entity_mapper.data_row:
                continue
            for item in self.itemize_valuestring_for_multiples(datarow[pvf]):
                if item != "":
                    trt.hook_prop_object(prop_type = concept, input_value = item)

        # metaprop territory represent events
        trt.hook_prop_object(prop_type = "C2286", input_value = entity_mapper.entity['id'])


        # genre relation, "genre_id":"C0335",
        for item in self.itemize_valuestring_for_multiples(datarow["genre_id"]):
            if item != "" in  item:
                if "C" in item:
                    trt.make_relation_record("Classification",trt.entity['id'], entity_mapper.get_entity_id(item))
                else:
                    logger.error(f"Creating territory through event generation:  did not get C entity for its classifications. Got '{item}'.")

        # audit record
        trt.create_audit_record(entity_id=trt.entity['id'], object=trt.entity)

        # check the ancestry #####################################################
        t_parts = trt.entity['legacyId'].split("-")
        size = len(t_parts)


        # save order
        trt.entity['data']['parent']['order'] = int(t_parts[-1])

        # if exists, connect, if doesnt, create, connect to root and connect the lowest part to the new middle
        if size == 3:
            checked_territory_lid = "-".join(t_parts[0:-1])
            # logger.info(f" Trying to go with {checked_territory_lid}.")
            if not self.checkTerritoryExists(checked_territory_lid): # territory does not exist
                trt_new = self.EMP.make(entity_name="ITerritory", data_row=datarow)
                trt_new.make_alive(legacyId= checked_territory_lid, label=t_parts[1], label_language=datarow['document_label_language'], register_id_where="texts")
                parent_id = trt_new.entity['id']

                checked_territory_lid = "-".join(t_parts[0:-2])
                if self.checkTerritoryExists(checked_territory_lid):
                    root_territory_id = entity_ids["texts"][checked_territory_lid]
                    trt_new.entity['data']['parent']['territoryId'] = root_territory_id
                    trt_new.entity['data']['parent']['order'] = int(t_parts[-2])
                else:
                    logger.error(f"The root territory {checked_territory_lid} does not exists.")

                # audit record
                trt_new.create_audit_record(entity_id=trt_new.entity['id'], object=trt_new.entity)
            else:
                parent_id =  entity_ids['texts'][checked_territory_lid]

            # territory does exist, connect the triggering territory to its parent
            trt.entity['data']['parent']['territoryId'] = parent_id
        elif size == 2:
            # just connect it to the supposedly existing root
            checked_territory_lid = "-".join(t_parts[0:1])
            if self.checkTerritoryExists(checked_territory_lid):
                root_territory_id = entity_ids["texts"][checked_territory_lid]
                trt.entity['data']['parent']['territoryId'] = root_territory_id
                trt.entity['data']['parent']['order'] = int(t_parts[-1])
            else:
                logger.error(f"The root territory {checked_territory_lid} does not exists.")


class LocationParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    def special_modern_name(self, operation, value, entity_mapper, field_name = "special_modern_name"):
        # "For non-empty, generate new L entities from this col.
        # 1) Attribute values: label = this col.; status = ""approved"", label_language = next col., entity_logical_type = same as col. D.
        # 2) Create ""identical entity"" relation between the Latin entity captured by this row (""id"") and this new entity.  Exact structure TBA (awaiting Adam) - until it is, create the relation as follows: append a metaprop to the original Latin location in ""id"", which will say: [""id"" Location] - has - C0691 identical entity - [modern location created from modern_name].)
        # 3) Attach ""class"" metaprop to this modern location from col. ""modern_name_class_id"" (described by the regular instructions)."

        origin = f"Making Location entity '{value}' from " + entity_mapper.data_row['legacyId'] + f" by field {field_name}."
        language = entity_mapper.data_row['modern_name_language']

        if value != "" and value!="NA" and value!="N/A" and value!="NS":  # check value
            label = value
            lentity = entity_mapper.make_lentity(label, origin=origin)
            lentity['language'] = entity_mapper.enum_mapper['language'][language]


            # create settlement prop OLD
            #  pt C0565  value in modern_name_class_id  OLD
            # prop_type = "C0565"
            # prop_value = entity_mapper.data_row['modern_name_class_id'] # another C
            #
            # for item in self.itemize_valuestring_for_multiples(prop_value, origin=origin):
            #     prop_type_id, prop_value_id = entity_mapper.prepare_prop_object_data(prop_type, item, origin=origin)
            #
            #     # make IProp object
            #     prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
            #     # append prop to the location entity
            #     lentity['props'].append(prop_object)

            # make classification for C0565 job
            # logger.info(f"{entity_mapper.data_row}")
            class_value = entity_mapper.data_row['modern_name_class_id'] # another C
            for item in self.itemize_valuestring_for_multiples(class_value, origin=origin):
                entity_mapper.make_relation_record('Classification', lentity['id'], entity_mapper.get_entity_id(item))


            # make identity relation
            # {type: RelationType.Identification, entityIds: [Location 1 id, Location 2 id], certainty: [Certainty.Certain]},
            # {type: "I", entityIds: [Location 1 id, Location 2 id], certainty: "1"},
            entity_mapper.make_relation_identity_record(entity_mapper.entity['id'], lentity['id'], "1")

            # create audit record
            entity_mapper.create_audit_record(entity_id=lentity['id'], object=lentity)



# Run

In [44]:
relation_records = [] # null relation records
audits = [] # null audits records
additional_entities = [] # null additional entities records
# making empty table of additional entities
tables['values'] = pd.DataFrame(columns=['id','value','origin'])
tables['locations'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['events'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['props'] = pd.DataFrame(columns=['id','type','type_id','value','value_id','original_field','origin','entityId','legacyId'])
tables['relations'] = pd.DataFrame(columns=['id','type','logic','certainty','entityIds','origin'])

logger.info(f"Start")
#cp = ParseController(entity_list=['texts',"concepts"])

# cp = ParseController(entity_list=["R0075_persons","R0035_locations","R0083_events"])
#cp = ParseController(entity_list=["R0035_locations","R0083_events"])
#cp = ParseController(entity_list=["texts"])

cp = ParseController(entity_list=['texts','actions', 'manuscripts','resources','concepts','R0006_persons', 'R0007_locations','R0008_events',"R0075_persons","R0035_locations","R0083_events"])

# cp = ParseController(entity_list=['texts','actions', 'manuscripts','resources','concepts','R0006_persons', 'R0007_locations','R0008_events',"R0075_persons","R0035_locations","R0083_events"], existing_parser=cp, reparse_entity_list=['R0035_locations'])

# cp = ParseController(entity_list=['texts', "concepts", "R0008_events"])
#cp = ParseController(entity_list=[ "R0007_locations"])
# , 'R0006_persons', 'R0007_locations'
# cp = ParseController(entity_list=['texts','manuscripts','resources','concepts','actions','R0006_persons', 'R0007_locations']) # , 'R0006_person \as', 'R0007_locations'
# cp = ParseController(entity_list=['R0006_persons', 'R0007_locations'])
#cp = ParseController(entity_list=['R0008_events'])
#cp = ParseController(entity_list=['actions','concepts'])
cp.parse()
logger.info(f"End")

2022-11-25 19:25:49,489 INFO Start
2022-11-25 19:25:49,492 INFO TEXTS Uncertain parsing instructions in 0 columns: .
2022-11-25 19:25:49,512 INFO TEXTS 64 columns have been dropped (discard:64, unknown:0: []). Table now has 52 columns, inside:9,propvalue:14, special:5, proptype: 4, proptype_2nd: 0, dependent:0, reference_part:0. Originally 116 columns.
2022-11-25 19:25:49,515 INFO ACTIONS Uncertain parsing instructions in 0 columns: .
2022-11-25 19:25:49,541 INFO ACTIONS 63 columns have been dropped (discard:63, unknown:0: []). Table now has 25 columns, inside:14,propvalue:0, special:1, proptype: 0, proptype_2nd: 0, dependent:0, reference_part:0. Originally 88 columns.
2022-11-25 19:25:49,543 INFO MANUSCRIPTS Uncertain parsing instructions in 0 columns: .
2022-11-25 19:25:49,557 INFO MANUSCRIPTS 35 columns have been dropped (discard:35, unknown:0: []). Table now has 31 columns, inside:8,propvalue:8, special:4, proptype: 4, proptype_2nd: 0, dependent:0, reference_part:0. Originally 66 c

### Merging reciprocal relation records
Antonym, PropertyReciprocal, SubjectActant1Reciprocal, Synonym, Related

In [45]:
# ANT, PRR, SAR, SYN, REL
def getE(eIds):
    eids = eIds.copy() # has to be copy, otherwise it modifies original relation_records list-arrays
    eids.sort()
    return eids[0]+"!"+eids[1]

rdf = pd.DataFrame(relation_records)
rdf['eids'] = rdf['entityIds'].apply(lambda row:getE(row))
removeRR  = rdf[rdf[['type','eids']].duplicated()].sort_values('eids')['id'].to_list()
removeRR

['3246e328-c4cb-4d29-af0d-9e5d792bf3b9',
 '621d223b-bf8e-4e01-8c52-fdf06c75102a',
 'c7143ff9-6750-437d-91cc-364673d62293',
 '72bce0ac-0f47-47a9-ab0c-4b941cad545f',
 '47f6067b-6c18-42e6-8f72-13d54a6a7aca',
 '71943206-f4b8-42a7-8eef-8db22b64a33d',
 'd721e478-301d-4268-ae01-85c2a78f58ee',
 'f582e6bf-0a9b-43c2-805d-1293c88f65aa',
 '3d4eb64c-4a17-4f4e-bf73-d6c370e9d2ba',
 'cfbf5db3-2edb-4562-9be8-2a3473f62f98',
 '7698fa1a-fc5f-40b0-9697-b2d541d930cf',
 'c903bebd-5fa8-4064-90dd-55c750ad5d5b',
 '32039aee-31e5-4433-a87e-44e8866763a9',
 '4cede5b2-10e8-468f-865d-faae7f1033b9',
 '1e8e2a81-1de8-4917-b218-9ba0f460afee',
 '9ef18b43-b773-4d62-8824-dd0839d6c195',
 'b207a7cd-5c6e-4c31-9dd9-fefeb3ff545a',
 '205d1cff-aaa1-4773-a41b-167468874ced',
 '9d6c697f-b6d4-4bbb-9d80-7c60fbcf20db',
 'aafb46cd-3eb8-4e87-b18b-1d5dbfccd197',
 'd1828d94-658c-4d7d-8500-549b81b7848e',
 'cef7f45c-b3b6-457f-88cf-78e64319409f',
 '807a4de7-ca60-467d-a095-e10220846513',
 'd6eb7345-0f03-4ab6-ab6b-837b0809f20f',
 '933e2181-a296-

In [46]:
relation_records_cleaned = []
for item in relation_records:
    if item['id'] not in removeRR:
        relation_records_cleaned.append(item)

In [47]:
len(relation_records), len(relation_records_cleaned)

(8465, 8019)

In [40]:
# check for relations entityIds
# a=0
# for item in relation_records_cleaned:
#     #if "81e38bb2-6f84-4b30-acac-b6931929dbaf" == item['id']:
#     if "7db804db-fa2d-4624-810d-b3440e2f4001" in item['entityIds'][0] and item['type']=="SCL":
#         a += 1
#         print(item)
# print(a)

{'id': 'dbb07e20-a1c6-49a0-a47e-dfde606f8e88', 'type': 'SCL', 'entityIds': ['7db804db-fa2d-4624-810d-b3440e2f4001', '479cc4f3-b877-44a5-a0dd-c6a7cc813163']}
1


### Profiling

In [ ]:
#%reload_ext line_profiler

# def profile():
#    cp.parsers['concepts'].parse_rows()

#%lprun -f EntityMapper.get_entity_id profile()

In [ ]:
# %prun -s tottime profile()

In [ ]:
# %lprun?

In [19]:
tables['events']

,id,value,origin,legacyId
0,d6c9ac84-fd4a-4a0b-8c7a-070026f3892a,Creation of ‘Acta Concilii Tolosani’,special T153>:creation_event_id E0001_T153,E0001_T153
1,d2f728b1-e641-4a15-94ff-1f8856da868c,Creation of ‘Acta Sancti Officii Bononie ab an...,special T77>:creation_event_id E0002_T77,E0002_T77
2,71a84efe-379f-43ac-822a-9e35aa983fd7,Creation of ‘Acta Synodi Atrebatensis’,special T94>:creation_event_id E0003_T94,E0003_T94
3,b1dce1bb-496b-4849-badd-e0b0bd2e1172,Creation of ‘Actus pontificum Cenomannis in ur...,special T131>:creation_event_id E0004_T131,E0004_T131
4,76b773b8-9a6a-4409-b145-aefd83a70099,Creation of ‘Annales Aquenses’,special T141>:creation_event_id E0005_T141,E0005_T141
...,...,...,...,...
290,9549cd46-8108-4d4f-ae41-7ed8eabd1cb2,"Creation of Paris, Bibliothèque nationale de F...",special>:creation_event_id E0139_M157,E0139_M157
291,c7318df0-d957-479f-8857-0309f5449479,"Creation of Paris, Bibliothèque nationale de F...",special>:creation_event_id E0140_M158,E0140_M158
292,ecacf90c-aee9-4132-acf7-0f5637aed58a,Creation of MSS containing the confirmation of...,special>:creation_event_id E0141_M159,E0141_M159
293,27231056-1320-424b-97c1-d04426b3f90a,Creation of MSS with Millau court records conc...,special>:creation_event_id E0142_M160,E0142_M160


In [167]:
# what is this d72b2bee-47b4-4abc-8c88-793c72ab676d ?

# {'parent': {'territoryId': 'eac54f75-822d-4c41-a995-c41ee4d1b8fc', 'order': 0}}

pd.DataFrame(additional_entities)[pd.DataFrame(additional_entities)['id']=="d98d04ac-850a-4274-a292-2210e6628afd"]


,class,id,label,language,detail,data,props,notes,status,references,legacyId


### Last checks

In [28]:
print(json.dumps(cp.parsers['actions'].js_objects[0]))

{"class": "A", "id": "157b6fdc-4acc-4890-a392-cfac0611dd1c", "legacyId": "A0168", "label": "abiurationem recepit", "language": "lat", "detail": "received abjuration (from sb)", "data": {"entities": {"a1": ["P", "G"], "a2": ["NULL"], "s": ["P", "G"]}, "valencies": {"a1": "\"a\" + 6 | 2", "a2": "NULL", "s": "1"}}, "props": [{"bundleEnd": false, "bundleStart": false, "certainty": "1", "children": [], "elvl": "1", "id": "026a0219-5ba9-404b-9296-076c653f9291", "logic": "1", "mood": [], "moodvariant": "1", "bundleOperator": "a", "type": {"entityId": "2fde570a-f104-4cd2-9b1b-b2d7f08efe5d", "elvl": "1", "logic": "1", "partitivity": "1", "virtuality": "1"}, "value": {"entityId": "2dc77198-ca35-47cb-a2e9-62276131c4c4", "elvl": "1", "logic": "1", "partitivity": "1", "virtuality": "1"}}, {"bundleEnd": false, "bundleStart": false, "certainty": "1", "children": [], "elvl": "1", "id": "6c5509f6-1f75-49a0-a516-d73073bb42eb", "logic": "1", "mood": [], "moodvariant": "1", "bundleOperator": "a", "type": 

In [20]:
logger.info(f"There are {len(additional_entities)} additionally created entities (e.g. values, resources ...).")

2022-11-25 14:00:08,721 INFO There are 3920 additionally created entities (e.g. values, resources ...).


# Output the parsed data

In [48]:
# root data
root_territory = """
 {
    "id": "T0",
    "class": "T",
    "data": { "parent": false },
    "label": "root",
    "detail": "",
    "language": "lat",
    "notes": [],
    "props": []
  }
"""

In [49]:
# connects outcomes from the individual parsers
cp.load_json_objects()

# save json object list of the parsed entities from google sheets in the text file
with open('../datasets/all-test/new_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(cp.js_objects, f)

# save json object list of the "newly created entities" in the text file
with open('../datasets/all-test/additional_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(additional_entities, f)

# read  entities.json
with open('../datasets/all/entities.json','r') as f:
    #entities_content = f.readlines()
    entities_content = f.read().replace('\n', '')

# read  entities.json
with open('../datasets/all-test/new_entities.json','r') as f:
    #entities_content = f.readlines()
    new_entities_content = f.read().replace('\n', '')

additional_entities_string = json.dumps(additional_entities)

# write new and combine with old test data
with open('../datasets/all-test/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content = entities_content[0:-1] +", " + additional_entities_string[1:-1]+ ", " + new_entities_content[1:]
    f.write(str(merge_content))

# write just the new parse data to the entities json.
with open('../datasets/all-parsed/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content =  "[" + root_territory + "," + additional_entities_string[1:-1]+ ", " + new_entities_content[1:]
    f.write(str(merge_content))


# write relations
with open('../datasets/all-parsed/relations.json','w', encoding='utf-8') as f:
    json.dump(relation_records_cleaned, f)

# write the audits json.
with open('../datasets/all-parsed/audits.json','w', encoding='utf-8') as f:
    json.dump(audits, f)


logger.info("END")

2022-11-25 19:33:45,423 INFO END


In [23]:
len(relation_records), len(relation_records_cleaned)

(8465, 8019)

In [222]:
additional_entities


[{'class': 'E',
  'id': '85643e42-212f-4d00-afd8-5e0e98bea243',
  'label': 'Creation of ‘Process against Bernard Niort and his family’',
  'language': 'eng',
  'detail': '',
  'data': {'logicalType': '1'},
  'props': [{'bundleEnd': False,
    'bundleStart': False,
    'certainty': '1',
    'children': [],
    'elvl': '1',
    'id': '9c044b74-1e31-47ff-9028-89dd438edb32',
    'logic': '1',
    'mood': [],
    'moodvariant': '1',
    'bundleOperator': 'a',
    'type': {'entityId': '375d1b8c-e2b8-4e58-9d5c-cefd82e0fb14',
     'elvl': '1',
     'logic': '1',
     'partitivity': '1',
     'virtuality': '1'},
    'value': {'entityId': '3275ce56-d96e-4f3b-9347-755554def6cd',
     'elvl': '1',
     'logic': '1',
     'partitivity': '1',
     'virtuality': '1'}}],
  'notes': ['Import batch [development] 2022-10-20 08:55:10.784252',
   'special T1>:creation_event_id E0001_T1'],
  'status': '1',
  'references': [],
  'legacyId': 'E0001_T1'},
 {'class': 'V',
  'id': '3275ce56-d96e-4f3b-9347-755554

In [ ]:
# Log tables google sheet
# https://docs.google.com/spreadsheets/d/1UpbJckLrRYTCz7wcEYhhcc-ihJxiUENx1unCCQqE4TE/edit#gid=0


# d.open_gsheet("logtables","https://docs.google.com/spreadsheets/d/1UpbJckLrRYTCz7wcEYhhcc-ihJxiUENx1unCCQqE4TE")

# d.write_df_to_gsheet("logtables", "values", tables["values"])
# d.write_df_to_gsheet("logtables", "resources", tables["resources"])
# d.write_df_to_gsheet("logtables", "locations", tables["locations"])
# d.write_df_to_gsheet("logtables", "events", tables["events"])
# d.write_df_to_gsheet("logtables", "props", tables["props"])



In [59]:
d.open_gsheet("discouraged","https://docs.google.com/spreadsheets/d/10x8TV2OmkcGt6zHw-3B-I3BhonETa0iQV424vCB9qSQ/edit#gid=0")

20 2022-10-30 10:16:53 : Opening gsheet handle discouraged


In [66]:
discouraged_concepts = tables['concepts'][tables['concepts']['status']=="discouraged"][['legacyId','label']]
discouraged_concepts_ids_list = discouraged_concepts['legacyId'].to_list()
discouraged_concepts_ids_list

['C2090',
 'C0408',
 'C0410',
 'C3069',
 'C0602',
 'C0165',
 'C0394',
 'C0541',
 'C0653',
 'C0744',
 'C0395',
 'C0417',
 'C2776',
 'C0125',
 'C0231',
 'C0611',
 'C0612',
 'C0683',
 'C0764',
 'C0784',
 'C0433',
 'C0636',
 'C0701',
 'C1271',
 'C1272',
 'C1273',
 'C0449',
 'C0455',
 'C0457',
 'C0531',
 'C0475',
 'C0672',
 'C0432',
 'C0439',
 'C0441',
 'C0673',
 'C0494',
 'C0495',
 'C0205',
 'C0206',
 'C0207',
 'C0498',
 'C0505',
 'C0518',
 'C0519',
 'C0520',
 'C2087',
 'C0203',
 'C0496',
 'C0497',
 'C0502',
 'C0503',
 'C0421',
 'C0422',
 'C0174',
 'C0176',
 'C0183',
 'C0184',
 'C0175',
 'C0177',
 'C2091',
 'C0210',
 'C0499',
 'C0500',
 'C1112',
 'C0628',
 'C0630',
 'C0180',
 'C0219',
 'C0525',
 'C0529',
 'C0627',
 'C0629',
 'C0238',
 'C0278',
 'C0515',
 'C0516',
 'C0115',
 'C0179',
 'C0240',
 'C0030',
 'C0924',
 'C0925',
 'C0926',
 'C0622',
 'C0249',
 'C0208',
 'C0028',
 'C0029',
 'C2096',
 'C0489',
 'C1158',
 'C1169',
 'C2600',
 'C2412',
 'C0023',
 'C0025',
 'C2168',
 'C0710',
 'C0443',


In [69]:
discouraged_concepts.set_index('legacyId').loc['C3324']['label']

'house of origin'

In [73]:
found = [('table','legacyId','column_with_concept','discouraged_concept_id','discouraged_concept_label')]

for t, tt in tables.items():
    #df = tables['resources']
    df = tables[t]
    #mask = np.column_stack([df[col].str.contains(r"C00", na=False) for col in df])
    mask = np.column_stack([df[col].isin(discouraged_concepts_ids_list) for col in df])
    result = df.loc[mask.any(axis=1)]
    if not result.empty:
        #found[t] = []
        print(t)
        lids = result['legacyId'].to_list()
        print(lids)
        for l in lids:
            target = df[df['legacyId'] == l]
            for c in target.columns:
                #print(target[c].iloc[0])
                if c=="legacyId":
                    continue
                if t=="concepts":
                    if target['status'].iloc[0] == "discouraged":
                        continue
                if target[c].iloc[0] in discouraged_concepts_ids_list:
                    found.append((t,l,c, target[c].iloc[0],discouraged_concepts.set_index('legacyId').loc[target[c].iloc[0]]['label']))



actions
['A0182', 'A0415', 'A0320', 'A0030', 'A0104']
concepts
['C2090', 'C0408', 'C0409', 'C0410', 'C0948', 'C0949', 'C0950', 'C3069', 'C3052', 'C0602', 'C0165', 'C0394', 'C0541', 'C0653', 'C0744', 'C0395', 'C0392', 'C0731', 'C2003', 'C0416', 'C0417', 'C2776', 'C0125', 'C0231', 'C0611', 'C0612', 'C0683', 'C0764', 'C0784', 'C0415', 'C0433', 'C0636', 'C0806', 'C0160', 'C0647', 'C0701', 'C1230', 'C1231', 'C1271', 'C1272', 'C1273', 'C0335', 'C0194', 'C0376', 'C0449', 'C0454', 'C0455', 'C0456', 'C0457', 'C0476', 'C0530', 'C0531', 'C0474', 'C0475', 'C0672', 'C0217', 'C0432', 'C0438', 'C0439', 'C0440', 'C0441', 'C0673', 'C0860', 'C0863', 'C0864', 'C1811', 'C0822', 'C1975', 'C2774', 'C0494', 'C0495', 'C0205', 'C0206', 'C0207', 'C0498', 'C0505', 'C0518', 'C0519', 'C0520', 'C2087', 'C0203', 'C0496', 'C0497', 'C0502', 'C0503', 'C0421', 'C0422', 'C0174', 'C0176', 'C0183', 'C0184', 'C0175', 'C0177', 'C2091', 'C0210', 'C0499', 'C0500', 'C1112', 'C0628', 'C0630', 'C0180', 'C0219', 'C0525', 'C0529', 

In [74]:
rdf = pd.DataFrame(found)
new_header = rdf.iloc[0] #grab the first row for the header
rdf = rdf[1:] #take the data less the header row
rdf.columns = new_header
rdf

,table,legacyId,column_with_concept,discouraged_concept_id,discouraged_concept_label
1,actions,A0182,related_entity_id,C0165,extra tempus gracie
2,actions,A0415,event_equivalent_superclass3_id,C2412,act
3,actions,A0320,event_equivalent_id,C1042,confessio
4,actions,A0030,subject_semantic_id,C1465,adversary
5,actions,A0030,actant1_semantic_id,C1465,adversary
...,...,...,...,...,...
209,concepts,C3207,plural_id,C1161,idiote
210,R0035_locations,L0003_R0035,localisation_precision_id,C3314,ambiguous
211,R0035_locations,L0027_R0035,localisation_precision_id,C3314,ambiguous
212,R0035_locations,L0075_R0035,class_id,C0851,locus


In [75]:
d.write_df_to_gsheet("discouraged", "discouraged_concepts", rdf)

20 2022-10-30 10:26:30 : Trying to write target dataset 'discouraged_concepts' to gsheet handle: discouraged
20 2022-10-30 10:26:30 : Using existing output worksheet
20 2022-10-30 10:26:31 : Successfully written to gsheet handle: discouraged


In [ ]:
#mask = np.column_stack([df[col].str.contains(r"\^", na=False) for col in df])
#df.loc[mask.any(axis=1)]
